In [5]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
%load_ext google.cloud.bigquery
from google.cloud import bigquery

client=bigquery.Client()
project_id = "som-nero-phi-jonc101"

/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


# Base Cohort 2024

In [2]:
query ="""

# select all blood culture orders
with base as (
SELECT DISTINCT
        op.anon_id, 
        op.pat_enc_csn_id_coded,
        op.order_proc_id_coded, 
        op.order_time_jittered_utc,
        EXTRACT(YEAR FROM op.order_time_jittered_utc) as order_year ,  
        op.ordering_mode,
        op.department_id,
    
    FROM 
        `som-nero-phi-jonc101.shc_core_2024.order_proc` op
     INNER JOIN
        `som-nero-phi-jonc101.shc_core_2024.lab_result` lr
    ON
        op.order_proc_id_coded = lr.order_id_coded
    WHERE
        op.order_type LIKE "Microbiology%"
        AND op.description LIKE "%BLOOD%"
        AND not op.order_status like any ('Discontinued','Canceled')
), 
# filter for ED orders
ED_orders as(
select 
b.anon_id,
b.pat_enc_csn_id_coded,
b.order_proc_id_coded,
b.order_time_jittered_utc,
b.order_year,
b.ordering_mode,
b.department_id,
department_name,
from base b inner join `som-nero-phi-jonc101.shc_core_2024.dep_map` using(department_id)
where lower(department_name) like '%emergency%'
group by b.anon_id,b.pat_enc_csn_id_coded,b.order_proc_id_coded,b.order_time_jittered_utc,b.order_year,b.ordering_mode,b.department_id,department_name
)
# make sure all the orders are from this ED encounter
SELECT 
  b.anon_id,
  b.pat_enc_csn_id_coded,
  b.order_proc_id_coded,
  b.order_time_jittered_utc as blood_culture_order_datetime_utc,
  b.order_year,
  ad.effective_time_jittered_utc as ed_arrival_datetime_utc,
FROM 
  ED_orders b
INNER JOIN 
  `som-nero-phi-jonc101.shc_core_2024.adt` ad
USING 
  (anon_id,pat_enc_csn_id_coded,department_id)
Where event_type='Admission'
and b.order_time_jittered_utc>=ad.effective_time_jittered_utc # confirmed that each admission encounter has only one admission time and 
GROUP BY b.anon_id,b.pat_enc_csn_id_coded,b.order_proc_id_coded,b.order_time_jittered_utc,b.order_year,ad.effective_time_jittered_utc
"""
base_cohort = client.query(query).to_dataframe()


In [3]:
# # Upload refactor_base_cohort to BigQuery as a temporary table
table_id_base_cohort_temp = f"{project_id}.blood_culture_stewardship_sandy_2024.base_cohort_temp"

# Upload the DataFrame to BigQuery
base_cohort.to_gbq(
    destination_table=table_id_base_cohort_temp,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded base_cohort to {table_id_base_cohort_temp}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/3413686401.py:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  base_cohort.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 2245.34it/s]

Uploaded base_cohort to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.base_cohort_temp


In [2]:
table_id_base_cohort_temp = f"{project_id}.blood_culture_stewardship_sandy_2024.base_cohort_temp"
base_cohort = client.query(f"""
select * from {table_id_base_cohort_temp}
""").to_dataframe()


In [3]:
base_cohort[["anon_id","pat_enc_csn_id_coded","order_proc_id_coded"]].nunique()

anon_id                  73287
pat_enc_csn_id_coded    117800
order_proc_id_coded     216464
dtype: int64

# Label

In [ ]:
# # Old
# query = f"""
# With culture as ( 
#     SELECT DISTINCT
#          c.anon_id,
#          c.pat_enc_csn_id_coded,
#          c.order_proc_id_coded,
#          lr.ord_value,
#          coalesce(lr.extended_value_comment,lr.extended_comp_comment) as comment,
#     FROM 
#         `{table_id_base_cohort_temp}` c
#     INNER JOIN
#         `som-nero-phi-jonc101.shc_core_2024.lab_result` lr
#        on c.anon_id=lr.anon_id 
#           and c.order_proc_id_coded=lr.order_id_coded
#           and c.pat_enc_csn_id_coded=lr.pat_enc_csn_id_coded
#           and c.blood_culture_order_datetime_utc=lr.order_time_jittered_utc
#     ) ,
#     # confimred that could be multiple rows (multiple organisms growth from lab_result table for each blood culture order

# culture_growth as (
# select op.anon_id,
#        op.pat_enc_csn_id_coded,
#        op.order_proc_id_coded,
#        1 as was_pos,
# from culture op  
#                 INNER join (SELECT DISTINCT anon_id,order_proc_id_coded, organism
#                         FROM `som-nero-phi-jonc101.shc_core_2024.culture_sensitivity`
#                         where organism is not null) cs USING (anon_id,order_proc_id_coded) 
# WHERE not lower(ord_value) like any ("%no%grow%","%not%detect%","negative")
#      AND not upper(comment) like any ('%NO%GROWTH%','%COAG%NEG%STAPH%','%GRAM%+%RODS%',
#                                       '%GRAM%POS%RODS%','%CONTAMIN%')
# )
# select distinct c.*, # i fixed this distinct , otherwise we have duplciate of 185924
# case when (cg.was_pos=1) then cg.was_pos else 0 end as positive_blood_culture 
# from `{table_id_base_cohort_temp}` c LEFT JOIN culture_growth cg 
# using(anon_id,pat_enc_csn_id_coded,order_proc_id_coded)
# """


# # Execute the query
# base_cohort_with_labels = client.query(query).to_dataframe()
# print(f"Created base_cohort_with_labels with {len(base_cohort_with_labels)} rows")

Created refactor_cohort_with_labels with 216464 rows


In [ ]:
# prevalence before any exlucsion 
# old
print("the prevalence of positive blood culture before any exclusion is", round(base_cohort_with_labels["positive_blood_culture"].mean() * 100, 2), "%")

the prevalence of positive blood culture before any exclusion is 2.96 %


In [ ]:
# # new 
# query = f"""
# WITH culture AS (
#   SELECT DISTINCT
#     c.anon_id,
#     c.pat_enc_csn_id_coded,
#     c.order_proc_id_coded,
#     lr.ord_value,
#     COALESCE(lr.extended_value_comment, lr.extended_comp_comment) AS comment
#   FROM `{table_id_base_cohort_temp}` c
#   INNER JOIN `som-nero-phi-jonc101.shc_core_2024.lab_result` lr
#     ON c.anon_id = lr.anon_id
#    AND c.order_proc_id_coded = lr.order_id_coded
#    AND c.pat_enc_csn_id_coded = lr.pat_enc_csn_id_coded
#    AND c.blood_culture_order_datetime_utc = lr.order_time_jittered_utc
# ),

# -- this can create multiple rows per order (multiple organisms)
# raw_culture_growth AS (
#   SELECT
#     cu.anon_id,
#     cu.pat_enc_csn_id_coded,
#     cu.order_proc_id_coded,
#     1 AS was_pos
#   FROM culture cu
#   INNER JOIN (
#     SELECT DISTINCT anon_id, order_proc_id_coded, organism
#     FROM `som-nero-phi-jonc101.shc_core_2024.culture_sensitivity`
#     WHERE organism IS NOT NULL
#   ) cs
#     USING (anon_id, order_proc_id_coded)
#   WHERE NOT (
#       LOWER(cu.ord_value) LIKE '%no%grow%' OR
#       LOWER(cu.ord_value) LIKE '%not%detect%' OR
#       LOWER(cu.ord_value) LIKE '%negative%'
#     )
#     AND NOT (
#       UPPER(cu.comment) LIKE '%NO%GROWTH%' OR
#       UPPER(cu.comment) LIKE '%COAG%NEG%STAPH%' OR
#       UPPER(cu.comment) LIKE '%GRAM%+%RODS%' OR
#       UPPER(cu.comment) LIKE '%GRAM%POS%RODS%' OR
#       UPPER(cu.comment) LIKE '%CONTAMIN%'
#     )
# ),

# -- collapse to 1 row per order
# culture_growth AS (
#   SELECT
#     anon_id,
#     pat_enc_csn_id_coded,
#     order_proc_id_coded,
#     MAX(was_pos) AS was_pos
#   FROM raw_culture_growth
#   GROUP BY 1,2,3
# )

# SELECT
#   c.*,
#   COALESCE(cg.was_pos, 0) AS positive_blood_culture
# FROM `{table_id_base_cohort_temp}` c
# LEFT JOIN culture_growth cg
#   USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded)
# """

# # Execute the query
# base_cohort_with_labels = client.query(query).to_dataframe()
# print(f"Created base_cohort_with_labels with {len(base_cohort_with_labels)} rows")

Created base_cohort_with_labels with 216464 rows


In [4]:
# new v2

query = f"""
WITH culture AS (
  SELECT DISTINCT
    c.anon_id,
    c.pat_enc_csn_id_coded,
    c.order_proc_id_coded,
    lr.ord_value,
    COALESCE(lr.extended_value_comment, lr.extended_comp_comment) AS comment
  FROM {table_id_base_cohort_temp} c
  JOIN `som-nero-phi-jonc101.shc_core_2024.lab_result` lr
    ON c.anon_id = lr.anon_id
   AND c.order_proc_id_coded = lr.order_id_coded
   AND c.pat_enc_csn_id_coded = lr.pat_enc_csn_id_coded
  #  AND c.blood_culture_order_datetime_utc = lr.order_time_jittered_utc
),

-- only orders that actually had an organism in sensitivity
culture_with_sens AS (
  SELECT
    cu.anon_id,
    cu.pat_enc_csn_id_coded,
    cu.order_proc_id_coded,
    cu.ord_value,
    cu.comment
  FROM culture cu
  JOIN (
    SELECT DISTINCT anon_id, order_proc_id_coded
    FROM `som-nero-phi-jonc101.shc_core_2024.culture_sensitivity`
    WHERE organism IS NOT NULL
  ) cs
  USING (anon_id, order_proc_id_coded)
),

-- flag each row
row_flags AS (
  SELECT
    anon_id,
    pat_enc_csn_id_coded,
    order_proc_id_coded,
    ord_value,
    comment,

    -- base contaminant / no-growth signals
    LOWER(ord_value) LIKE '%no%grow%'         AS is_no_growth,
    LOWER(ord_value) LIKE '%not%detect%'      AS is_not_detected,
    LOWER(ord_value) LIKE '%negative%'        AS is_negative_text,
    REGEXP_CONTAINS(
      UPPER(comment),
      r'NO GROWTH|COAG\s*NEG\s*STAPH|GRAM\+\s*RODS|GRAM\s*POS\s*RODS|CONTAMIN'
    ) AS is_comment_contam,

    -- the two single-bottle CoNS patterns
    REGEXP_CONTAINS(
      LOWER(ord_value),
      r'^aerobic bottle:\s*coag negative staphylococcus'
    ) AS is_aerobic_cons,

    REGEXP_CONTAINS(
      LOWER(ord_value),
      r'^anaerobic bottle:\s*coag negative staphylococcus'
    ) AS is_anaerobic_cons,

    -- your special case: "Both Aerobic bottles: Coag Negative Staphylococcus" -> POSITIVE
    REGEXP_CONTAINS(
      LOWER(ord_value),
      r'^both aerobic bottles:\s*coag negative staphylococcus'
    ) AS is_both_aerobic_bottles_cons

  FROM culture_with_sens
),

-- now collapse to order-level
order_flags AS (
  SELECT
    anon_id,
    pat_enc_csn_id_coded,
    order_proc_id_coded,

    -- did we see that special "both aerobic bottles ..." row?
    MAX(CASE WHEN is_both_aerobic_bottles_cons THEN 1 ELSE 0 END) AS has_both_aerobic_bottles_cons,

    -- did we see aerobic CoNS?
    MAX(CASE WHEN is_aerobic_cons THEN 1 ELSE 0 END) AS has_aerobic_cons,

    -- did we see anaerobic CoNS?
    MAX(CASE WHEN is_anaerobic_cons THEN 1 ELSE 0 END) AS has_anaerobic_cons,

    -- did we see ANY row that is NOT obviously no-growth/negative/comment-contam
    -- and also not one of the single-bottle CoNS?
    MAX(
      CASE
        WHEN is_both_aerobic_bottles_cons THEN 1            -- this one counts as positive
        WHEN is_aerobic_cons OR is_anaerobic_cons THEN 0    -- we'll handle with bottle-pair logic
        WHEN is_no_growth OR is_not_detected OR is_negative_text OR is_comment_contam THEN 0
        ELSE 1
      END
    ) AS has_other_positive_like
  FROM row_flags
  GROUP BY 1,2,3
),

-- final decision per order
labeled_orders AS (
  SELECT
    anon_id,
    pat_enc_csn_id_coded,
    order_proc_id_coded,
    CASE
      -- 1) special text: "Both Aerobic bottles: Coag Negative Staphylococcus"
      WHEN has_both_aerobic_bottles_cons = 1 THEN 1

      -- 2) we saw BOTH aerobic and anaerobic single-bottle CoNS -> call it positive
      WHEN has_aerobic_cons = 1 AND has_anaerobic_cons = 1 THEN 1

      -- 3) any other clearly positive-like row we didn't filter away
      WHEN has_other_positive_like = 1 THEN 1

      -- 4) otherwise, it's negative (e.g. only aerobic CoNS, or only anaerobic CoNS, or just no growth)
      ELSE 0
    END AS positive_blood_culture
  FROM order_flags
)

SELECT
  b.*,
  IFNULL(l.positive_blood_culture, 0) AS positive_blood_culture
FROM {table_id_base_cohort_temp} b
LEFT JOIN labeled_orders l
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded);

"""
base_cohort_with_labels = client.query(query).to_dataframe()
print(f"Created base_cohort_with_labels with {len(base_cohort_with_labels)} rows")


Created base_cohort_with_labels with 216464 rows


In [5]:
# prevalence before any exlucsion 
# new 

print("the prevalence of positive blood culture before any exclusion is", round(base_cohort_with_labels["positive_blood_culture"].mean() * 100, 2), "%")

the prevalence of positive blood culture before any exclusion is 4.14 %


In [7]:
# Upload refactor_base_cohort to BigQuery as a temporary table
table_id_cohort_with_labels_temp = f"{project_id}.blood_culture_stewardship_sandy_2024.base_cohort_with_labels_temp"

# Upload the DataFrame to BigQuery
base_cohort_with_labels.to_gbq(
    destination_table=table_id_cohort_with_labels_temp,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded base_cohort_with_labels to {table_id_cohort_with_labels_temp}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/255363665.py:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  base_cohort_with_labels.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 1972.86it/s]

Uploaded base_cohort_with_labels to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.base_cohort_with_labels_temp


# exclusion 2: no positive blood culture within preceding 14 days


In [6]:
tcol = 'blood_culture_order_datetime_utc'
df = base_cohort_with_labels.copy()

# 1. Normalize and sort (positives first if tie — optional)
df[tcol] = pd.to_datetime(df[tcol], utc=True, errors='coerce')\
              .dt.tz_convert('UTC').dt.tz_localize(None)
df = (
    df.dropna(subset=[tcol])
      .sort_values(['anon_id', tcol, 'positive_blood_culture'],
                   ascending=[True, True, False],
                   kind='mergesort')
)

# 2. Mark positive times
pos_time = df[tcol].where(df['positive_blood_culture'].eq(1))

# 3. For each patient, carry forward the most recent positive *up to previous row*
latest_pos = pos_time.groupby(df['anon_id'], sort=False).ffill()

# 4. Compute timedelta difference
delta = df[tcol] - latest_pos

# 5. Drop if strictly after (Δ > 0) and within 14 days
drop_mask = delta.gt(pd.Timedelta(0)) & delta.le(pd.Timedelta(days=14))

# 6. Keep others
base_cohort_with_labels_exclusion_2 = df.loc[~drop_mask, [
    'anon_id','pat_enc_csn_id_coded','order_proc_id_coded',
    'blood_culture_order_datetime_utc','order_year',
    'ed_arrival_datetime_utc','positive_blood_culture'
]].reset_index(drop=True)
# dropped orders = those that had a prior positive within 14 days
base_cohort_with_labels_exclusion_2_dropped = df.loc[drop_mask, [
    'anon_id','pat_enc_csn_id_coded','order_proc_id_coded',
    'blood_culture_order_datetime_utc','order_year',
    'ed_arrival_datetime_utc','positive_blood_culture'
]].reset_index(drop=True)



In [7]:
base_cohort_with_labels_exclusion_2

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,order_year,ed_arrival_datetime_utc,positive_blood_culture
0,JC1000129,131354606122,876959985,2023-04-23 22:44:00,2023,2023-04-23 22:41:00+00:00,1
1,JC1000129,131354606122,876959986,2023-04-23 22:44:00,2023,2023-04-23 22:41:00+00:00,0
2,JC1000159,131233687107,528085198,2017-06-25 16:42:00,2017,2017-06-25 16:38:00+00:00,1
3,JC1000159,131233687107,528085197,2017-06-25 16:42:00,2017,2017-06-25 16:38:00+00:00,0
4,JC1000413,131018846539,401262600,2012-04-20 05:09:00,2012,2012-04-20 04:39:00+00:00,0
...,...,...,...,...,...,...,...
214472,JC999695,131029726519,434470924,2014-01-27 18:45:00,2014,2014-01-27 18:12:00+00:00,0
214473,JC999695,131029726519,434470925,2014-01-27 18:45:00,2014,2014-01-27 18:12:00+00:00,0
214474,JC999695,131047054269,448446821,2014-09-17 06:38:00,2014,2014-09-17 05:30:00+00:00,0
214475,JC999695,131047054269,448459871,2014-09-17 08:46:00,2014,2014-09-17 05:30:00+00:00,0


In [26]:
random_id = base_cohort_with_labels_exclusion_2[base_cohort_with_labels_exclusion_2["positive_blood_culture"] == 1]["anon_id"].sample(1).iloc[0]

base_cohort_with_labels_exclusion_2[base_cohort_with_labels_exclusion_2["anon_id"] == random_id]
# JC732487


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,order_year,ed_arrival_datetime_utc,positive_blood_culture
164696,JC614323,131238991423,537292696,2017-10-10 19:53:00,2017,2017-10-10 19:48:00+00:00,1
164697,JC614323,131238991423,537292697,2017-10-10 19:53:00,2017,2017-10-10 19:48:00+00:00,0


In [ ]:
base_cohort_with_labels_exclusion_2_dropped[base_cohort_with_labels_exclusion_2_dropped["anon_id"] == random_id]

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,order_year,ed_arrival_datetime_utc,positive_blood_culture


In [107]:
base_cohort_with_labels_exclusion_2["order_year"].value_counts().sort_index()

order_year
2011     2981
2012     5149
2013     9447
2014    10593
2015    10424
2016    10526
2017    10517
2018    15440
2019    18115
2020    18862
2021    19794
2022    21944
2023    28468
2024    27701
2025     4516
Name: count, dtype: Int64

In [29]:
print("after exclusion 2, the prevalence of positive blood culture is", round(base_cohort_with_labels_exclusion_2["positive_blood_culture"].mean() * 100, 2), "%")

after exclusion 2, the prevalence of positive blood culture is 4.17 %


# overwrite the order within the same encounter

In [30]:
# Apply to your kept dataframe (e.g., `result` from earlier)
base_cohort_with_labels_exclusion_2_overwrite = base_cohort_with_labels_exclusion_2.copy()

# Ensure 0/1 integers
# df2['positive_blood_culture'] = df2['positive_blood_culture'].fillna(0).astype(int)

# Overwrite: if any positive within the same (anon_id, encounter), set all to 1
base_cohort_with_labels_exclusion_2_overwrite['positive_blood_culture'] = (
    base_cohort_with_labels_exclusion_2_overwrite.groupby(['anon_id', 'pat_enc_csn_id_coded'])['positive_blood_culture']
       .transform('max')
    .astype(int)
)

In [31]:
print("after exclusion 2 and overwrite, the prevalence of positive blood culture is", round(base_cohort_with_labels_exclusion_2_overwrite["positive_blood_culture"].mean() * 100, 2), "%")
print("the number of valid orders is", len(base_cohort_with_labels_exclusion_2_overwrite))

after exclusion 2 and overwrite, the prevalence of positive blood culture is 7.52 %
the number of valid orders is 214477


In [32]:
encounter_level = base_cohort_with_labels_exclusion_2_overwrite[["anon_id", "pat_enc_csn_id_coded", "positive_blood_culture"]].drop_duplicates(subset=["anon_id", "pat_enc_csn_id_coded"])
print("after exclusion 2 and overwrite, the prevalence of positive blood culture at encounter level is", round(encounter_level["positive_blood_culture"].mean() * 100, 2), "%")
print("the number of valid encounter is", len(encounter_level))

after exclusion 2 and overwrite, the prevalence of positive blood culture at encounter level is 7.32 %
the number of valid encounter is 117194


In [33]:
# Upload refactor_base_cohort to BigQuery as a temporary table
table_id_base_cohort_with_labels_exlucsion_n_overwrite = f"{project_id}.blood_culture_stewardship_sandy_2024.base_cohort_with_labels_exlucsion_n_overwrite"

# Upload the DataFrame to BigQuery
base_cohort_with_labels_exclusion_2_overwrite.to_gbq(
    destination_table=table_id_base_cohort_with_labels_exlucsion_n_overwrite,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded base_cohort_with_labels_exclusion_2_overwrite to {table_id_base_cohort_with_labels_exlucsion_n_overwrite}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/1846847501.py:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  base_cohort_with_labels_exclusion_2_overwrite.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 2545.09it/s]

Uploaded base_cohort_with_labels_exclusion_2_overwrite to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.base_cohort_with_labels_exlucsion_n_overwrite


In [36]:
base_cohort_with_labels_exclusion_2_overwrite

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,order_year,ed_arrival_datetime_utc,positive_blood_culture
0,JC1000129,131354606122,876959985,2023-04-23 22:44:00,2023,2023-04-23 22:41:00+00:00,1
1,JC1000129,131354606122,876959986,2023-04-23 22:44:00,2023,2023-04-23 22:41:00+00:00,1
2,JC1000159,131233687107,528085198,2017-06-25 16:42:00,2017,2017-06-25 16:38:00+00:00,1
3,JC1000159,131233687107,528085197,2017-06-25 16:42:00,2017,2017-06-25 16:38:00+00:00,1
4,JC1000413,131018846539,401262600,2012-04-20 05:09:00,2012,2012-04-20 04:39:00+00:00,0
...,...,...,...,...,...,...,...
214472,JC999695,131029726519,434470924,2014-01-27 18:45:00,2014,2014-01-27 18:12:00+00:00,0
214473,JC999695,131029726519,434470925,2014-01-27 18:45:00,2014,2014-01-27 18:12:00+00:00,0
214474,JC999695,131047054269,448446821,2014-09-17 06:38:00,2014,2014-09-17 05:30:00+00:00,0
214475,JC999695,131047054269,448459871,2014-09-17 08:46:00,2014,2014-09-17 05:30:00+00:00,0


# Demo
age, sex, ADI Score, bmi

### filter out < 18 years old

In [34]:
query = f"""
with demos as (
    SELECT c.*,
           demo.gender,
           demo.canonical_race as race,
           demo.bmi as bmi,
          case when demo.birth_date_jittered is not null then DATE_DIFF(CAST(c.blood_culture_order_datetime_utc AS DATE), CAST(demo.birth_date_jittered_utc AS DATE), YEAR)
                else null end AS age,
    FROM 
        `{table_id_base_cohort_with_labels_exlucsion_n_overwrite}` c
    LEFT JOIN 
        `som-nero-phi-jonc101.shc_core_2024.demographic` demo
    ON 
        c.anon_id = demo.anon_id
)
select * from demos where (age is null or age >18)
"""
final_base_bmi_age_race_gender= client.query(query).to_dataframe()

In [35]:
print("the number of valid orders (>18 years old) is", len(final_base_bmi_age_race_gender))
print("the number of valid encounter (>18 years old) is", len(final_base_bmi_age_race_gender["pat_enc_csn_id_coded"].unique()))

the number of valid orders (>18 years old) is 198015
the number of valid encounter (>18 years old) is 102015


### adi calculation
- 9 digit exact match
- 5 digit imputed with CA state avg match
- null value imputed with CA state avg match

In [37]:
from google.cloud import bigquery

PROJECT_ID = "som-nero-phi-jonc101"
client = bigquery.Client(project=PROJECT_ID)

# Base cohort table (already defined in your session)
TABLE_BASE = table_id_base_cohort_with_labels_exlucsion_n_overwrite

# Reference tables
TABLE_ZIP = f"`{PROJECT_ID}.shc_core_2024.zip`"
TABLE_ADI = f"`{PROJECT_ID}.mapdata.ADI_data_CA`"

query = f"""
WITH
-- 1) Cohort + cleaned ZIP (remove dashes)
cohort_zip AS (
  SELECT
    mc.*,
    REPLACE(z.zip, '-', '') AS zip_clean
  FROM {TABLE_BASE} mc
  LEFT JOIN {TABLE_ZIP} z
    ON mc.anon_id = z.anon_id
),

-- 2) Keep only numeric ADI scores (drop flags like 'P','U','NA', etc.)
adi_numeric AS (
  SELECT
    zip,
    CAST(adi_score AS FLOAT64) AS adi_num
  FROM {TABLE_ADI}
  WHERE SAFE_CAST(adi_score AS FLOAT64) IS NOT NULL
),

-- 3) ZIP5 average from ZIP+4 rows (your colleague's method)
zip5_avg AS (
  SELECT LEFT(zip, 5) AS zip5, AVG(adi_num) AS avg_adi_zip5
  FROM adi_numeric
  WHERE LENGTH(zip) = 9
  GROUP BY zip5
),

-- 4) Statewide fallback computed from ZIP5 means
state_avg AS (
  SELECT AVG(avg_adi_zip5) AS avg_adi_state
  FROM zip5_avg
),

-- 5) Score with priority: exact ZIP -> ZIP5 avg -> STATE avg
scored AS (
  SELECT
    cz.*,
    COALESCE(
      adi_exact.adi_num,        -- exact ZIP match
      z5.avg_adi_zip5,          -- ZIP5 mean
      sa.avg_adi_state          -- statewide mean of ZIP5 means
    ) AS adi_score,
    CASE
      WHEN adi_exact.adi_num IS NOT NULL THEN 0
      ELSE 1
    END AS adi_imputed_flag
  FROM cohort_zip cz
  LEFT JOIN adi_numeric adi_exact
    ON cz.zip_clean = adi_exact.zip
  LEFT JOIN zip5_avg z5
    ON LEFT(cz.zip_clean, 5) = z5.zip5
  CROSS JOIN state_avg sa
)

-- Final output
SELECT *
FROM scored
"""

job = client.query(query, location="US")
adi_score_df = job.to_dataframe()


In [38]:
final_base_bmi_age_race_gender_adi = pd.merge(final_base_bmi_age_race_gender, adi_score_df[["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "blood_culture_order_datetime_utc","zip_clean", "adi_score", "adi_imputed_flag"]], on=["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "blood_culture_order_datetime_utc"], how="left")

In [40]:
# Upload refactor_base_cohort to BigQuery as a temporary table
table_id_final_base_bmi_age_race_gender_adi_temp = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_bmi_age_race_gender_adi_temp"

# Upload the DataFrame to BigQuery
final_base_bmi_age_race_gender_adi.to_gbq(
    destination_table=table_id_final_base_bmi_age_race_gender_adi_temp,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_bmi_age_race_gender_adi to {table_id_final_base_bmi_age_race_gender_adi_temp}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/3892412909.py:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_bmi_age_race_gender_adi.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 2763.05it/s]

Uploaded final_base_bmi_age_race_gender_adi to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_bmi_age_race_gender_adi_temp


In [ ]:
# query = f"""
# select * from blood_culture_stewardship_sandy_2024.final_base_bmi_age_race_gender_adi_temp
# """
# final_base_bmi_age_race_gender_adi = client.query(query).to_dataframe()


# Check for organism prevalence

In [6]:
table_id_final_base_bmi_age_race_gender_adi_temp = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_bmi_age_race_gender_adi_temp"


In [12]:
query = f"""
SELECT DISTINCT 
    c.anon_id, 
    c.pat_enc_csn_id_coded, 
    order_proc_id_coded, 
    blood_culture_order_datetime_utc, 
    order_year, 
    positive_blood_culture, 
    organism 
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c 
INNER JOIN `som-nero-phi-jonc101.shc_core_2024.culture_sensitivity`
    USING (anon_id, order_proc_id_coded)
WHERE organism IS NOT NULL
ORDER BY anon_id, order_proc_id_coded
"""
organism_prevalence = client.query(query).to_dataframe()

In [16]:
organism_prevalence.groupby("organism")["organism"].count().size()

TypeError: 'int' object is not callable

# Vitals

## temp, resp, heart rate,  blood pressure, SPO2
### temp: 26.5 - 45 degree C
### resp ranges 0-90(peds), adult: 0-60 
### hr: 250 max
### bp: 250 (sys) /150(dia)
### O2: N/A

In [ ]:
# # refactored old:
# # this very prelim codespace:
# query = f"""
# -- Min/Avg/Max (+ median/mode) vitals per blood culture order in [-24h, 0h]
# WITH normalized AS (
#   SELECT
#     c.anon_id,
#     c.pat_enc_csn_id_coded,
#     c.order_proc_id_coded,
#     c.blood_culture_order_datetime_utc,
#     f.recorded_time_jittered_utc AS ts,
#     UPPER(TRIM(f.row_disp_name)) AS row_name,
#     f.meas_value
#   FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS c
#   LEFT JOIN `som-nero-phi-jonc101.shc_core_2024.flowsheet` AS f
#     ON c.anon_id = f.anon_id
#    AND TIMESTAMP_DIFF(
#          f.recorded_time_jittered_utc,
#          CAST(c.blood_culture_order_datetime_utc AS TIMESTAMP),
#          HOUR
#        ) BETWEEN -24 AND 0
# ),
# labeled AS (
#   SELECT
#     anon_id,
#     pat_enc_csn_id_coded,
#     order_proc_id_coded,
#     blood_culture_order_datetime_utc,
#     ts,
#     CASE
#       WHEN row_name LIKE '%TEMP%' THEN 'temp'
#       WHEN row_name LIKE '%RESP%' THEN 'resp'
#       WHEN row_name IN ('PULSE','HEART RATE') OR row_name LIKE '%HEART RATE%' THEN 'heart_rate'
#       WHEN row_name IN ('BP','NIBP') THEN 'blood_pressure'
#       WHEN row_name LIKE '%SPO2%' OR row_name LIKE '%O2%' THEN 'o2_saturation'
#       ELSE NULL
#     END AS vital,
#     meas_value
#   FROM normalized
#   -- do NOT filter here; preserves orders with no vitals
# ),
# parsed AS (
#   -- numeric parsing; BP -> val1=sys, val2=dia; others -> val1
#   SELECT
#     anon_id,
#     pat_enc_csn_id_coded,
#     order_proc_id_coded,
#     blood_culture_order_datetime_utc,
#     ts,
#     vital,
#     CASE
#       WHEN vital = 'blood_pressure' THEN
#         SAFE_CAST(SPLIT(REGEXP_REPLACE(meas_value, r'[^0-9/]', ''), '/')[SAFE_OFFSET(0)] AS FLOAT64)
#       ELSE SAFE_CAST(meas_value AS FLOAT64)
#     END AS val1,
#     CASE
#       WHEN vital = 'blood_pressure' THEN
#         SAFE_CAST(SPLIT(REGEXP_REPLACE(meas_value, r'[^0-9/]', ''), '/')[SAFE_OFFSET(1)] AS FLOAT64)
#       ELSE NULL
#     END AS val2
#   FROM labeled
#   -- do NOT filter here; preserves orders with no vitals
# )
# SELECT
#   anon_id,
#   pat_enc_csn_id_coded,
#   order_proc_id_coded,
#   blood_culture_order_datetime_utc,

#   -- temperature
#   MIN(IF(vital='temp', val1, NULL)) AS temp_min,
#   AVG(IF(vital='temp', val1, NULL)) AS temp_avg,
#   MAX(IF(vital='temp', val1, NULL)) AS temp_max,
#   APPROX_QUANTILES(IF(vital='temp', val1, NULL), 100)[OFFSET(50)] AS temp_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 1) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM parsed p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'temp'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS temp_mode,

#   -- respiration rate
#   MIN(IF(vital='resp', val1, NULL)) AS resp_min,
#   AVG(IF(vital='resp', val1, NULL)) AS resp_avg,
#   MAX(IF(vital='resp', val1, NULL)) AS resp_max,
#   APPROX_QUANTILES(IF(vital='resp', val1, NULL), 100)[OFFSET(50)] AS resp_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM parsed p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'resp'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS resp_mode,


#   -- heart rate
#   MIN(IF(vital='heart_rate', val1, NULL)) AS hr_min,
#   AVG(IF(vital='heart_rate', val1, NULL)) AS hr_avg,
#   MAX(IF(vital='heart_rate', val1, NULL)) AS hr_max,
#   APPROX_QUANTILES(IF(vital='heart_rate', val1, NULL), 100)[OFFSET(50)] AS hr_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM parsed p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'heart_rate'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS hr_mode,


#   -- blood pressure (systolic)
#   MIN(IF(vital='blood_pressure', val1, NULL)) AS sysbp_min,
#   AVG(IF(vital='blood_pressure', val1, NULL)) AS sysbp_avg,
#   MAX(IF(vital='blood_pressure', val1, NULL)) AS sysbp_max,
#   APPROX_QUANTILES(IF(vital='blood_pressure', val1, NULL), 100)[OFFSET(50)] AS sysbp_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM parsed p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'blood_pressure'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS sysbp_mode,

#   -- blood pressure (diastolic)
#   MIN(IF(vital='blood_pressure', val2, NULL)) AS diabp_min,
#   AVG(IF(vital='blood_pressure', val2, NULL)) AS diabp_avg,
#   MAX(IF(vital='blood_pressure', val2, NULL)) AS diabp_max,
#   APPROX_QUANTILES(IF(vital='blood_pressure', val2, NULL), 100)[OFFSET(50)] AS diabp_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val2, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM parsed p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'blood_pressure'
#         AND p2.val2 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS diabp_mode,

#   -- SpO2
#   MIN(IF(vital='o2_saturation', val1, NULL)) AS spo2_min,
#   AVG(IF(vital='o2_saturation', val1, NULL)) AS spo2_avg,
#   MAX(IF(vital='o2_saturation', val1, NULL)) AS spo2_max,
#   APPROX_QUANTILES(IF(vital='o2_saturation', val1, NULL), 100)[OFFSET(50)] AS spo2_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM parsed p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'o2_saturation'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS spo2_mode


# FROM parsed
# GROUP BY
#   anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc
# ORDER BY
#   anon_id, blood_culture_order_datetime_utc;
# """
# final_base_vitals = client.query(query).to_dataframe()


In [2]:
table_id_final_base_bmi_age_race_gender_adi_temp = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_bmi_age_race_gender_adi_temp"

In [ ]:
# #### V2 refactor after old code
# query = f"""
# -- Min/Avg/Max (+ median/mode) vitals per blood culture order in [-24h, 0h]
# -- Temperature is normalized (°C) from multiple displays/units; others are placeholders (to be upgraded later).
# WITH flowsheet_window AS (
#   SELECT
#     c.anon_id,
#     c.pat_enc_csn_id_coded,
#     c.order_proc_id_coded,
#     c.blood_culture_order_datetime_utc,
#     f.recorded_time_jittered_utc AS ts,
#     TRIM(f.row_disp_name) AS row_name,
#     CAST(f.meas_value AS STRING) AS meas_value_str,
#     f.units AS units
#   FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS c
#   LEFT JOIN `som-nero-phi-jonc101.shc_core_2024.flowsheet` AS f
#     ON c.anon_id = f.anon_id
#    AND TIMESTAMP_DIFF(
#          f.recorded_time_jittered_utc,
#          CAST(c.blood_culture_order_datetime_utc AS TIMESTAMP),
#          HOUR
#        ) BETWEEN -24 AND 0
# ),

# -- Identify vitals (keep broad for future expansion)
# labeled AS (
#   SELECT
#     anon_id,
#     pat_enc_csn_id_coded,
#     order_proc_id_coded,
#     blood_culture_order_datetime_utc,
#     ts,
#     row_name,
#     meas_value_str,
#     units,
#     CASE
#       -- --- Temperature buckets (expandable) ---
#       WHEN row_name IN (
#         'ECMO Venous Blood Temperature',
#         'Rectal Temperature',
#         'Patient Core Temperature',
#         'ECMO Arterial Blood Temperature',
#         'Temperature',
#         'Temperature (C)',
#         'Temperature (Blood - PA line)',
#         'Temp',
#         'Temp (in Celsius)'
#       ) THEN 'temp'
#       -- --- Resp / HR / BP / SpO2 (placeholders; will upgrade later) ---
#       WHEN upper(trim(row_name)) LIKE '%RESP%' 
#       and not lower(trim(row_name)) like "%apnea%"
#       and not lower(trim(row_name)) like "%>%" 
#       and not lower(trim(row_name)) like "%corresponding inr%"THEN 'resp'
#       WHEN upper(trim(row_name)) IN ('PULSE','HEART RATE')
#         OR upper(trim(row_name)) LIKE '%HEART RATE%' THEN 'heart_rate'
#       WHEN upper(trim(row_name)) IN ('BP','NIBP') THEN 'blood_pressure'
#       WHEN upper(trim(row_name))  IN ('SPO2','O2', "O2 SAT") THEN 'o2_saturation'
#       # LIKE '%SPO2%' OR upper(trim(row_name)) LIKE '%O2%' THEN 'o2_saturation'
#       ELSE NULL
#     END AS vital
#   FROM flowsheet_window
# ),

# -- Parse numerics + detect temperature unit using:
# -- 1) proper spelling in units, 2) contains 'f'/'c' anywhere in units, 3) numeric inference
# parsed AS (
#   SELECT
#     anon_id,
#     pat_enc_csn_id_coded,
#     order_proc_id_coded,
#     blood_culture_order_datetime_utc,
#     ts,
#     vital,
#     row_name,
#     meas_value_str,
#     units,
#     -- numeric from value string (e.g., '98.6 F', '37 C')
#     COALESCE(
#       SAFE_CAST(meas_value_str AS FLOAT64),
#       SAFE_CAST(REGEXP_EXTRACT(meas_value_str, r'-?\\d+(?:\\.\\d+)?') AS FLOAT64)
#     ) AS value_num,

#     -- Step 1: proper spellings (highest priority)
#     CASE
#       WHEN REGEXP_CONTAINS(LOWER(CAST(units AS STRING)),
#            r'(?:\\b|°)\\s*f\\b|\\bdeg(?:ree|rees)?\\s*f\\b|\\bfahrenheit\\b') THEN 'F'
#       WHEN REGEXP_CONTAINS(LOWER(CAST(units AS STRING)),
#            r'(?:\\b|°)\\s*c\\b|\\bdeg(?:ree|rees)?\\s*c\\b|\\bcelsius\\b') THEN 'C'
#       ELSE NULL
#     END AS unit_proper
#   FROM labeled
#   -- keep nulls to preserve orders with no vitals
# ),

# unit_text_choice AS (
#   SELECT
#     anon_id,
#     pat_enc_csn_id_coded,
#     order_proc_id_coded,
#     blood_culture_order_datetime_utc,
#     ts,
#     vital,
#     row_name,
#     meas_value_str,
#     units,
#     value_num,
#     unit_proper,

#     -- Step 2: contains 'f' or 'c' anywhere in units (only if proper not found).
#     CASE
#       WHEN unit_proper IS NOT NULL THEN unit_proper
#       WHEN REGEXP_CONTAINS(LOWER(CAST(units AS STRING)), r'f')
#            AND NOT REGEXP_CONTAINS(LOWER(CAST(units AS STRING)), r'c') THEN 'F'
#       WHEN REGEXP_CONTAINS(LOWER(CAST(units AS STRING)), r'c')
#            AND NOT REGEXP_CONTAINS(LOWER(CAST(units AS STRING)), r'f') THEN 'C'
#       -- if both or neither appear, defer to numeric inference
#       ELSE NULL
#     END AS unit_from_text_any
#   FROM parsed
# ),

# canon AS (
#   SELECT
#     anon_id,
#     pat_enc_csn_id_coded,
#     order_proc_id_coded,
#     blood_culture_order_datetime_utc,
#     ts,
#     vital,
#     row_name,
#     meas_value_str,
#     units,
#     value_num,

#     -- Step 3: final unit = text result (proper or contains-any) OR numeric inference
#     COALESCE(
#       unit_from_text_any,
#       CASE
#         WHEN value_num BETWEEN 80 AND 113  THEN 'F'
#         WHEN value_num BETWEEN 26.7 AND 45 THEN 'C'
#         ELSE NULL
#       END
#     ) AS unit_norm,

#     -- Temperature canonicalization (°C)
#     CASE
#       WHEN vital = 'temp' AND value_num IS NOT NULL THEN
#         CASE
#           WHEN COALESCE(
#                  unit_from_text_any,
#                  CASE
#                    WHEN value_num BETWEEN 80 AND 113  THEN 'F'
#                    WHEN value_num BETWEEN 26.7 AND 45 THEN 'C'
#                    ELSE NULL
#                  END
#                ) = 'F'
#             THEN (value_num - 32) * 5/9
#           ELSE value_num  -- 'C' or unknown (leave as-is)
#         END
#       ELSE NULL
#     END AS temp_c,
#     -- --- PLACEHOLDERS (raw numeric for now) ---
#     CASE
#       WHEN vital IN ('heart_rate', 'resp', 'o2_saturation') THEN SAFE_CAST(value_num AS FLOAT64)
#       WHEN vital = 'blood_pressure' THEN
#         SAFE_CAST(SPLIT(REGEXP_REPLACE(meas_value_str, r'[^0-9/]', ''), '/')[SAFE_OFFSET(0)] AS FLOAT64)
#       ELSE NULL
#     END AS val1,
#     CASE
#       WHEN vital = 'blood_pressure' THEN
#         SAFE_CAST(SPLIT(REGEXP_REPLACE(meas_value_str, r'[^0-9/]', ''), '/')[SAFE_OFFSET(1)] AS FLOAT64)
#       ELSE NULL
#     END AS val2
#   FROM unit_text_choice
# )

# SELECT
#   anon_id,
#   pat_enc_csn_id_coded,
#   order_proc_id_coded,
#   blood_culture_order_datetime_utc,

#   -- ================= TEMPERATURE (°C) =================
#   MIN(IF(vital='temp', temp_c, NULL)) AS temp_min_c,
#   AVG(IF(vital='temp', temp_c, NULL)) AS temp_avg_c,
#   MAX(IF(vital='temp', temp_c, NULL)) AS temp_max_c,
#   APPROX_QUANTILES(IF(vital='temp', temp_c, NULL), 100)[OFFSET(50)] AS temp_median_c,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(temp_c, 1) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM canon p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'temp'
#         AND p2.temp_c IS NOT NULL
#       GROUP BY val
#     )
#   ) AS temp_mode_c,

#   -- ================= RESP (placeholder: raw numeric) =================
#   MIN(IF(vital='resp', val1, NULL)) AS resp_min,
#   AVG(IF(vital='resp', val1, NULL)) AS resp_avg,
#   MAX(IF(vital='resp', val1, NULL)) AS resp_max,
#   APPROX_QUANTILES(IF(vital='resp', val1, NULL), 100)[OFFSET(50)] AS resp_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM canon p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'resp'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS resp_mode,

#   -- ================= HEART RATE (placeholder: raw numeric) =================
#   MIN(IF(vital='heart_rate', val1, NULL)) AS hr_min,
#   AVG(IF(vital='heart_rate', val1, NULL)) AS hr_avg,
#   MAX(IF(vital='heart_rate', val1, NULL)) AS hr_max,
#   APPROX_QUANTILES(IF(vital='heart_rate', val1, NULL), 100)[OFFSET(50)] AS hr_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM canon p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'heart_rate'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS hr_mode,

#   -- ================= BLOOD PRESSURE (placeholder: sys/diastolic split) =================
#   MIN(IF(vital='blood_pressure', val1, NULL)) AS sysbp_min,
#   AVG(IF(vital='blood_pressure', val1, NULL)) AS sysbp_avg,
#   MAX(IF(vital='blood_pressure', val1, NULL)) AS sysbp_max,
#   APPROX_QUANTILES(IF(vital='blood_pressure', val1, NULL), 100)[OFFSET(50)] AS sysbp_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM canon p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'blood_pressure'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS sysbp_mode,

#   MIN(IF(vital='blood_pressure', val2, NULL)) AS diabp_min,
#   AVG(IF(vital='blood_pressure', val2, NULL)) AS diabp_avg,
#   MAX(IF(vital='blood_pressure', val2, NULL)) AS diabp_max,
#   APPROX_QUANTILES(IF(vital='blood_pressure', val2, NULL), 100)[OFFSET(50)] AS diabp_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val2, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM canon p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'blood_pressure'
#         AND p2.val2 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS diabp_mode,

#   -- ================= SpO2 (placeholder: raw numeric) =================
#   MIN(IF(vital='o2_saturation', val1, NULL)) AS spo2_min,
#   AVG(IF(vital='o2_saturation', val1, NULL)) AS spo2_avg,
#   MAX(IF(vital='o2_saturation', val1, NULL)) AS spo2_max,
#   APPROX_QUANTILES(IF(vital='o2_saturation', val1, NULL), 100)[OFFSET(50)] AS spo2_median,
#   (
#     SELECT AS VALUE ARRAY_AGG(val ORDER BY cnt DESC, val LIMIT 1)[SAFE_OFFSET(0)]
#     FROM (
#       SELECT CAST(ROUND(val1, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
#       FROM canon p2
#       WHERE p2.anon_id = anon_id
#         AND p2.pat_enc_csn_id_coded = pat_enc_csn_id_coded
#         AND p2.order_proc_id_coded = order_proc_id_coded
#         AND p2.blood_culture_order_datetime_utc = blood_culture_order_datetime_utc
#         AND p2.vital = 'o2_saturation'
#         AND p2.val1 IS NOT NULL
#       GROUP BY val
#     )
#   ) AS spo2_mode

# FROM canon
# GROUP BY
#   anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc
# ORDER BY
#   anon_id, blood_culture_order_datetime_utc;
# """

# final_base_vitals = client.query(query).to_dataframe()

### temperature

In [41]:
# Read the CSV directly
csv_url = "/Users/sandychen/Downloads/_Temp_ and _Temp (in Celsius)_updated.csv"
df = pd.read_csv(csv_url)
df.rename(columns={'Unnamed: 1': 'label'}, inplace=True)
valid_template = df[df["label"] == "Yes"]

In [42]:
# Conservative physiologic range (°C). Adjust if needed.
TEMP_C_MIN = 26.7
TEMP_C_MAX = 45.0

tmpl_list = valid_template["template"].tolist()
if tmpl_list:
    template_where = "AND f.template IN (" + ", ".join([f"'{t}'" for t in tmpl_list]) + ")"
else:
    template_where = ""  # no filter if list is empty

query = f"""
-- Temperature (°C) statistics per blood culture order in [-48h, 0h]
-- Keeps all base rows; imputes out-of-range temps to 37°C prior to aggregation
WITH flowsheet_window AS (
  SELECT
    c.anon_id,
    c.pat_enc_csn_id_coded,
    c.order_proc_id_coded,
    c.blood_culture_order_datetime_utc,
    f.recorded_time_jittered_utc AS ts,
    TRIM(f.row_disp_name) AS row_name,
    CAST(f.meas_value AS STRING) AS meas_value_str,
    f.units AS units,
    f.template
  FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS c
  LEFT JOIN `som-nero-phi-jonc101.shc_core_2024.flowsheet` AS f
    ON c.anon_id = f.anon_id
   AND TIMESTAMP_DIFF(
         f.recorded_time_jittered_utc,
         CAST(c.blood_culture_order_datetime_utc AS TIMESTAMP),
         HOUR
       ) BETWEEN -48 AND 0
    {template_where}
),

labeled AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ts, row_name, meas_value_str, units, template,
    CASE
      WHEN row_name IN (
        'ECMO Venous Blood Temperature', 'Rectal Temperature', 'Patient Core Temperature',
        'ECMO Arterial Blood Temperature', 'Temperature', 'Temperature (C)',
        'Temperature (Blood - PA line)', 'Temp', 'Temp (in Celsius)'
      ) THEN 'temp'
      ELSE NULL
    END AS vital
  FROM flowsheet_window
  WHERE row_name IS NOT NULL
),

parsed AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ts, vital, row_name, meas_value_str, units, template,
    COALESCE(
      SAFE_CAST(meas_value_str AS FLOAT64),
      SAFE_CAST(REGEXP_EXTRACT(meas_value_str, r'-?\\d+(?:\\.\\d+)?') AS FLOAT64)
    ) AS value_num,
    CASE
      WHEN REGEXP_CONTAINS(LOWER(units), r'(?:°|deg|fahrenheit|\\bf\\b)') THEN 'F'
      WHEN REGEXP_CONTAINS(LOWER(units), r'(?:°|deg|celsius|\\bc\\b)') THEN 'C'
      ELSE NULL
    END AS unit_hint
  FROM labeled
  WHERE vital = 'temp'
),

-- Canonicalize to °C (raw converted value = temp_c)
canon AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ts, template,
    CASE
      WHEN COALESCE(
             unit_hint,
             CASE WHEN value_num BETWEEN 80 AND 113 THEN 'F'
                  WHEN value_num BETWEEN 26.7 AND 45 THEN 'C' END
           ) = 'F' THEN (value_num - 32) * 5/9
      ELSE value_num
    END AS temp_c
  FROM parsed
),

-- Impute out-of-range temps to 37°C; keep diagnostics
canon_imputed AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ts, template,
    temp_c,
    CASE
      WHEN temp_c IS NULL THEN NULL
      WHEN temp_c < {TEMP_C_MIN} OR temp_c > {TEMP_C_MAX} THEN NULL
      ELSE temp_c
    END AS temp_c_imp,
    -- diagnostics
    CASE
      WHEN temp_c IS NULL THEN FALSE
      WHEN temp_c < {TEMP_C_MIN} OR temp_c > {TEMP_C_MAX} THEN TRUE
      ELSE FALSE
    END AS was_imputed
  FROM canon
),

-- Aggregate per order using the imputed value
temp_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ARRAY_AGG(DISTINCT template IGNORE NULLS) AS template_list,
    COUNT(*) AS temp_obs_n,
    COUNTIF(temp_c IS NOT NULL) AS temp_raw_nonnull_n,
    COUNTIF(was_imputed) AS temp_imputed_n,
    MIN(temp_c_imp) AS temp_min_c,
    AVG(temp_c_imp) AS temp_avg_c,
    MAX(temp_c_imp) AS temp_max_c,
    APPROX_QUANTILES(temp_c_imp, 100)[OFFSET(50)] AS temp_median_c
  FROM canon_imputed
  GROUP BY 1,2,3,4
),

-- Mode (0.1°C bins) using imputed values
temp_hist AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    CAST(ROUND(temp_c_imp, 1) AS FLOAT64) AS val, COUNT(*) AS cnt
  FROM canon_imputed
  WHERE temp_c_imp IS NOT NULL
  GROUP BY 1,2,3,4,5
),

temp_mode AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    val AS temp_mode_c
  FROM (
    SELECT t.*,
           ROW_NUMBER() OVER (
             PARTITION BY anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc
             ORDER BY cnt DESC, val ASC
           ) AS rn
    FROM temp_hist t
  )
  WHERE rn = 1
)

SELECT
  b.anon_id,
  b.pat_enc_csn_id_coded,
  b.order_proc_id_coded,
  b.blood_culture_order_datetime_utc,
  a.template_list,
  a.temp_obs_n,
  a.temp_raw_nonnull_n,
  a.temp_imputed_n,
  a.temp_min_c,
  a.temp_avg_c,
  a.temp_max_c,
  a.temp_median_c,
  m.temp_mode_c
FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS b
LEFT JOIN temp_agg a
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
LEFT JOIN temp_mode m
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_temp_only = client.query(query).to_dataframe()


In [47]:
# Upload the DataFrame to BigQuery
table_id_final_base_temperature_only= f"{project_id}.blood_culture_stewardship_sandy_2024.table_id_final_base_temperature_only"

final_base_temp_only.to_gbq(
    destination_table=table_id_final_base_temperature_only,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_temp_only to {table_id_final_base_temperature_only}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/3412480659.py:4: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_temp_only.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 4609.13it/s]

Uploaded final_base_temp_only to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.table_id_final_base_temperature_only


#### resp

In [48]:
RESP_MIN = 0
RESP_MAX = 90

query_resp = f"""
WITH flowsheet_window AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    f.recorded_time_jittered_utc AS ts,
    TRIM(f.row_disp_name) AS row_name,
    CAST(f.meas_value AS STRING) AS meas_value_str,
    f.units AS units
  FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS c
  LEFT JOIN `som-nero-phi-jonc101.shc_core_2024.flowsheet` AS f
    ON c.anon_id = f.anon_id
   AND TIMESTAMP_DIFF(f.recorded_time_jittered_utc, CAST(c.blood_culture_order_datetime_utc AS TIMESTAMP), HOUR)
       BETWEEN -48 AND 0
),

labeled AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ts, row_name, meas_value_str, units,
    CASE
      WHEN UPPER(TRIM(row_name)) IN ('RESP', 'RESP RATE')
      or TRIM(row_name) = '(Retired) Resp Rate Total'
      THEN 'resp' ELSE NULL
    END AS vital
  FROM flowsheet_window
  WHERE row_name IS NOT NULL
),

parsed AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, ts,
    COALESCE(
      SAFE_CAST(meas_value_str AS FLOAT64),
      SAFE_CAST(REGEXP_EXTRACT(meas_value_str, r'-?\\d+(?:\\.\\d+)?') AS FLOAT64)
    ) AS resp_val
  FROM labeled
  WHERE vital = 'resp'
),

canon AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, ts,
    CASE
      WHEN resp_val IS NULL THEN NULL
      WHEN resp_val < {RESP_MIN} OR resp_val > {RESP_MAX} THEN NULL
      ELSE resp_val
    END AS resp_ok
  FROM parsed
),

resp_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    COUNT(*) AS resp_obs_n,
    COUNTIF(resp_ok IS NOT NULL) AS resp_nonnull_n,
    MIN(resp_ok) AS resp_min,
    AVG(resp_ok) AS resp_avg,
    MAX(resp_ok) AS resp_max,
    APPROX_QUANTILES(resp_ok, 100)[OFFSET(50)] AS resp_median
  FROM canon
  GROUP BY 1,2,3,4
),

resp_hist AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    CAST(ROUND(resp_ok, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
  FROM canon
  WHERE resp_ok IS NOT NULL
  GROUP BY 1,2,3,4,5
),

resp_mode AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, val AS resp_mode
  FROM (
    SELECT t.*,
           ROW_NUMBER() OVER (
             PARTITION BY anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc
             ORDER BY cnt DESC, val ASC
           ) rn
    FROM resp_hist t
  ) WHERE rn = 1
)

SELECT
  b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
  a.resp_obs_n, a.resp_nonnull_n,
  a.resp_min, a.resp_avg, a.resp_max, a.resp_median,
  m.resp_mode
FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` b
LEFT JOIN resp_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
LEFT JOIN resp_mode m USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_resp_only = client.query(query_resp).to_dataframe()


In [52]:
# Upload the DataFrame to BigQuery
table_id_final_base_resp_only= f"{project_id}.blood_culture_stewardship_sandy_2024.table_id_final_base_resp_only"

final_base_resp_only.to_gbq(
    destination_table=table_id_final_base_resp_only,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_resp_only to {table_id_final_base_resp_only}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/927075462.py:4: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_resp_only.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 2403.61it/s]

Uploaded final_base_resp_only to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.table_id_final_base_resp_only


#### heart rate

In [53]:
# --- Heart Rate extraction/aggregation in [-48h, 0h] w.r.t. blood culture order ---

# Conservative HR bounds (bpm). Tweak as needed (e.g., peds vs adults).
HR_MIN = 0
HR_MAX = 250

# Optional: match your temperature query’s template filter
# tmpl_list = valid_template["template"].tolist() if "valid_template" in globals() else []

# if tmpl_list:
#     template_where_hr = "AND f.template IN (" + ", ".join([f"'{t}'" for t in tmpl_list]) + ")"
# else:
#     template_where_hr = ""  # no filter if list is empty or unavailable

template_where_hr = ""
query_hr = f"""
WITH flowsheet_window AS (
  SELECT
    c.anon_id,
    c.pat_enc_csn_id_coded,
    c.order_proc_id_coded,
    c.blood_culture_order_datetime_utc,
    f.recorded_time_jittered_utc AS ts,
    TRIM(f.row_disp_name) AS row_name,
    CAST(f.meas_value AS STRING) AS meas_value_str,
    f.units AS units,
    f.template
  FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS c
  LEFT JOIN `som-nero-phi-jonc101.shc_core_2024.flowsheet` AS f
    ON c.anon_id = f.anon_id
   AND TIMESTAMP_DIFF(
         f.recorded_time_jittered_utc,
         CAST(c.blood_culture_order_datetime_utc AS TIMESTAMP),
         HOUR
       ) BETWEEN -48 AND 0
    {template_where_hr}
),

labeled AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ts, row_name, meas_value_str, units, template,
    CASE
      -- Common HR labels (feel free to expand)
      WHEN UPPER(TRIM(row_name)) IN ('HEART RATE','PULSE')
      THEN 'heart_rate'
      ELSE NULL
    END AS vital
  FROM flowsheet_window
  WHERE row_name IS NOT NULL
),

parsed AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, ts, template,
    COALESCE(
      SAFE_CAST(meas_value_str AS FLOAT64),
      SAFE_CAST(REGEXP_EXTRACT(meas_value_str, r'-?\\d+(?:\\.\\d+)?') AS FLOAT64)
    ) AS hr_val
  FROM labeled
  WHERE vital = 'heart_rate'
),

canon AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, ts, template,
    CASE
      WHEN hr_val IS NULL THEN NULL
      WHEN hr_val < {HR_MIN} OR hr_val > {HR_MAX} THEN NULL
      ELSE hr_val
    END AS hr_ok
  FROM parsed
),

hr_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ARRAY_AGG(DISTINCT template IGNORE NULLS) AS template_list,
    COUNT(*) AS hr_obs_n,
    COUNTIF(hr_ok IS NOT NULL) AS hr_nonnull_n,
    MIN(hr_ok) AS hr_min,
    AVG(hr_ok) AS hr_avg,
    MAX(hr_ok) AS hr_max,
    APPROX_QUANTILES(hr_ok, 100)[OFFSET(50)] AS hr_median
  FROM canon
  GROUP BY 1,2,3,4
),

hr_hist AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    CAST(ROUND(hr_ok, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
  FROM canon
  WHERE hr_ok IS NOT NULL
  GROUP BY 1,2,3,4,5
),

hr_mode AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    val AS hr_mode
  FROM (
    SELECT t.*,
           ROW_NUMBER() OVER (
             PARTITION BY anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc
             ORDER BY cnt DESC, val ASC
           ) AS rn
    FROM hr_hist t
  )
  WHERE rn = 1
)

SELECT
  b.anon_id,
  b.pat_enc_csn_id_coded,
  b.order_proc_id_coded,
  b.blood_culture_order_datetime_utc,
  a.template_list,
  a.hr_obs_n,
  a.hr_nonnull_n,
  a.hr_min,
  a.hr_avg,
  a.hr_max,
  a.hr_median,
  m.hr_mode
FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS b
LEFT JOIN hr_agg a
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
LEFT JOIN hr_mode m
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""

final_base_hr_only = client.query(query_hr).to_dataframe()


In [55]:
# Upload the DataFrame to BigQuery
table_id_final_base_hr_only= f"{project_id}.blood_culture_stewardship_sandy_2024.table_id_final_base_hr_only"

final_base_hr_only.to_gbq(
    destination_table=table_id_final_base_hr_only,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_hr_only to {table_id_final_base_hr_only}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/893166263.py:4: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_hr_only.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 11397.57it/s]

Uploaded final_base_hr_only to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.table_id_final_base_hr_only


#### bp

In [56]:
# --- Blood Pressure extraction/aggregation in [-48h, 0h] w.r.t. blood culture order ---

BP_SYS_MAX = 250
BP_DIA_MAX = 150

# # Optional template filter (match your temp query behavior)
# tmpl_list = valid_template["template"].tolist() if "valid_template" in globals() else []
# if tmpl_list:
#     template_where_bp = "AND f.template IN (" + ", ".join([f"'{t}'" for t in tmpl_list]) + ")"
# else:
#     template_where_bp = ""  # no filter if list is empty or unavailable

template_where_bp = ""
query_bp = f"""
WITH flowsheet_window AS (
  SELECT
    c.anon_id,
    c.pat_enc_csn_id_coded,
    c.order_proc_id_coded,
    c.blood_culture_order_datetime_utc,
    f.recorded_time_jittered_utc AS ts,
    TRIM(f.row_disp_name) AS row_name,
    CAST(f.meas_value AS STRING) AS meas_value_str,
    f.units AS units,
    f.template
  FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS c
  LEFT JOIN `som-nero-phi-jonc101.shc_core_2024.flowsheet` AS f
    ON c.anon_id = f.anon_id
   AND TIMESTAMP_DIFF(
         f.recorded_time_jittered_utc,
         CAST(c.blood_culture_order_datetime_utc AS TIMESTAMP),
         HOUR
       ) BETWEEN -48 AND 0
    {template_where_bp}
),

labeled AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ts, row_name, meas_value_str, units, template,
    CASE
      WHEN UPPER(TRIM(row_name)) IN ('BP','NIBP') THEN 'blood_pressure'
      ELSE NULL
    END AS vital
  FROM flowsheet_window
  WHERE row_name IS NOT NULL
),

-- Parse systolic/diastolic from strings like "120/70"; strip noise first
parsed AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, ts, template,
    SAFE_CAST(SPLIT(REGEXP_REPLACE(meas_value_str, r'[^0-9/]', ''), '/')[SAFE_OFFSET(0)] AS FLOAT64) AS sys_val,
    SAFE_CAST(SPLIT(REGEXP_REPLACE(meas_value_str, r'[^0-9/]', ''), '/')[SAFE_OFFSET(1)] AS FLOAT64) AS dia_val
  FROM labeled
  WHERE vital = 'blood_pressure'
),

-- Apply upper cutoffs only (per your spec). Values above cutoff become NULL.
canon AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, ts, template,
    CASE
      WHEN sys_val IS NULL THEN NULL
      WHEN sys_val <= {BP_SYS_MAX} THEN sys_val
      ELSE NULL
    END AS bp_sys_ok,
    CASE
      WHEN dia_val IS NULL THEN NULL
      WHEN dia_val <= {BP_DIA_MAX} THEN dia_val
      ELSE NULL
    END AS bp_dia_ok
  FROM parsed
),

bp_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ARRAY_AGG(DISTINCT template IGNORE NULLS) AS template_list,
    COUNT(*) AS bp_obs_n,
    COUNTIF(bp_sys_ok IS NOT NULL) AS sys_nonnull_n,
    COUNTIF(bp_dia_ok IS NOT NULL) AS dia_nonnull_n,
    MIN(bp_sys_ok) AS sysbp_min,
    AVG(bp_sys_ok) AS sysbp_avg,
    MAX(bp_sys_ok) AS sysbp_max,
    APPROX_QUANTILES(bp_sys_ok, 100)[OFFSET(50)] AS sysbp_median,
    MIN(bp_dia_ok) AS diabp_min,
    AVG(bp_dia_ok) AS diabp_avg,
    MAX(bp_dia_ok) AS diabp_max,
    APPROX_QUANTILES(bp_dia_ok, 100)[OFFSET(50)] AS diabp_median
  FROM canon
  GROUP BY 1,2,3,4
),

-- Modes (integer bins)
bp_sys_hist AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    CAST(ROUND(bp_sys_ok, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
  FROM canon
  WHERE bp_sys_ok IS NOT NULL
  GROUP BY 1,2,3,4,5
),
bp_dia_hist AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    CAST(ROUND(bp_dia_ok, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
  FROM canon
  WHERE bp_dia_ok IS NOT NULL
  GROUP BY 1,2,3,4,5
),
bp_sys_mode AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    val AS sysbp_mode
  FROM (
    SELECT t.*,
           ROW_NUMBER() OVER (
             PARTITION BY anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc
             ORDER BY cnt DESC, val ASC
           ) AS rn
    FROM bp_sys_hist t
  )
  WHERE rn = 1
),
bp_dia_mode AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    val AS diabp_mode
  FROM (
    SELECT t.*,
           ROW_NUMBER() OVER (
             PARTITION BY anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc
             ORDER BY cnt DESC, val ASC
           ) AS rn
    FROM bp_dia_hist t
  )
  WHERE rn = 1
)

SELECT
  b.anon_id,
  b.pat_enc_csn_id_coded,
  b.order_proc_id_coded,
  b.blood_culture_order_datetime_utc,
  a.template_list,
  a.bp_obs_n,
  a.sys_nonnull_n, a.dia_nonnull_n,
  a.sysbp_min, a.sysbp_avg, a.sysbp_max, a.sysbp_median,
  sm.sysbp_mode,
  a.diabp_min, a.diabp_avg, a.diabp_max, a.diabp_median,
  dm.diabp_mode
FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS b
LEFT JOIN bp_agg a
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
LEFT JOIN bp_sys_mode sm
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
LEFT JOIN bp_dia_mode dm
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""

final_base_bp_only = client.query(query_bp).to_dataframe()


In [57]:
# Upload the DataFrame to BigQuery
table_id_final_base_bp_only= f"{project_id}.blood_culture_stewardship_sandy_2024.table_id_final_base_bp_only"

final_base_bp_only.to_gbq(
    destination_table=table_id_final_base_bp_only,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_bp_only to {table_id_final_base_bp_only}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/459724529.py:4: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_bp_only.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 4951.95it/s]

Uploaded final_base_bp_only to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.table_id_final_base_bp_only


#### spo2

In [60]:
# --- SpO2 extraction/aggregation in [-48h, 0h] w.r.t. blood culture order ---

SPO2_MIN = 0     # percent
SPO2_MAX = 100   # percent

# # Optional: match your temperature query’s template filter
# tmpl_list = valid_template["template"].tolist() if "valid_template" in globals() else []
# if tmpl_list:
#     template_where_spo2 = "AND f.template IN (" + ", ".join([f"'{t}'" for t in tmpl_list]) + ")"
# else:
#     template_where_spo2 = ""  # no filter if list is empty or unavailable
template_where_spo2 = ""

query_spo2 = f"""
WITH flowsheet_window AS (
  SELECT
    c.anon_id,
    c.pat_enc_csn_id_coded,
    c.order_proc_id_coded,
    c.blood_culture_order_datetime_utc,
    f.recorded_time_jittered_utc AS ts,
    TRIM(f.row_disp_name) AS row_name,
    CAST(f.meas_value AS STRING) AS meas_value_str,
    f.units AS units,
    f.template
  FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS c
  LEFT JOIN `som-nero-phi-jonc101.shc_core_2024.flowsheet` AS f
    ON c.anon_id = f.anon_id
   AND TIMESTAMP_DIFF(
         f.recorded_time_jittered_utc,
         CAST(c.blood_culture_order_datetime_utc AS TIMESTAMP),
         HOUR
       ) BETWEEN -48 AND 0
    {template_where_spo2}
),

labeled AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ts, row_name, meas_value_str, units, template,
    CASE
      -- Common SpO2 aliases; exclude common non-SpO2 oxygen rows
      WHEN (
        UPPER(TRIM(row_name)) in ('SPO2', 'O2', 'O2 SAT')
      )
      THEN 'spo2'
      ELSE NULL
    END AS vital
  FROM flowsheet_window
  WHERE row_name IS NOT NULL
),

parsed AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, ts, template,
    -- extract numeric (e.g., "98 %", "96", etc.)
    COALESCE(
      SAFE_CAST(meas_value_str AS FLOAT64),
      SAFE_CAST(REGEXP_EXTRACT(meas_value_str, r'-?\\d+(?:\\.\\d+)?') AS FLOAT64)
    ) AS spo2_val
  FROM labeled
  WHERE vital = 'spo2'
),

canon AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc, ts, template,
    CASE
      WHEN spo2_val IS NULL THEN NULL
      WHEN spo2_val < {SPO2_MIN} OR spo2_val > {SPO2_MAX} THEN NULL
      ELSE spo2_val
    END AS spo2_ok
  FROM parsed
),

spo2_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ARRAY_AGG(DISTINCT template IGNORE NULLS) AS template_list,
    COUNT(*) AS spo2_obs_n,
    COUNTIF(spo2_ok IS NOT NULL) AS spo2_nonnull_n,
    MIN(spo2_ok) AS spo2_min,
    AVG(spo2_ok) AS spo2_avg,
    MAX(spo2_ok) AS spo2_max,
    APPROX_QUANTILES(spo2_ok, 100)[OFFSET(50)] AS spo2_median
  FROM canon
  GROUP BY 1,2,3,4
),

spo2_hist AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    CAST(ROUND(spo2_ok, 0) AS FLOAT64) AS val, COUNT(*) AS cnt
  FROM canon
  WHERE spo2_ok IS NOT NULL
  GROUP BY 1,2,3,4,5
),

spo2_mode AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    val AS spo2_mode
  FROM (
    SELECT t.*,
           ROW_NUMBER() OVER (
             PARTITION BY anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc
             ORDER BY cnt DESC, val ASC
           ) AS rn
    FROM spo2_hist t
  )
  WHERE rn = 1
)

SELECT
  b.anon_id,
  b.pat_enc_csn_id_coded,
  b.order_proc_id_coded,
  b.blood_culture_order_datetime_utc,
  a.template_list,
  a.spo2_obs_n,
  a.spo2_nonnull_n,
  a.spo2_min,
  a.spo2_avg,
  a.spo2_max,
  a.spo2_median,
  m.spo2_mode
FROM `{table_id_final_base_bmi_age_race_gender_adi_temp}` AS b
LEFT JOIN spo2_agg a
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
LEFT JOIN spo2_mode m
  USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""

final_base_spo2_only = client.query(query_spo2).to_dataframe()


In [61]:
# Upload the DataFrame to BigQuery
table_id_final_base_spo2_only= f"{project_id}.blood_culture_stewardship_sandy_2024.table_id_final_base_spo2_only"

final_base_spo2_only.to_gbq(
    destination_table=table_id_final_base_spo2_only,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_spo2_only to {table_id_final_base_spo2_only}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/3650448623.py:4: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_spo2_only.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 3177.50it/s]

Uploaded final_base_spo2_only to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.table_id_final_base_spo2_only


#### concat

In [63]:
import pandas as pd
from functools import reduce

# List of all per-vital DataFrames
dfs = [final_base_temp_only,final_base_resp_only, final_base_hr_only, final_base_bp_only, final_base_spo2_only]

# Merge keys
merge_keys = ["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "blood_culture_order_datetime_utc"]

# Keep only key columns + aggregate metrics (min, max, median, mode)
dfs_clean = [
    df.loc[:, df.columns.str.contains("|".join(merge_keys)) | df.columns.str.contains("(min|max|median|mode|avg)", case=False)]
    for df in dfs
]

# Merge all DataFrames on the same keys
final_base_vitals = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how="outer"), dfs_clean)


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/648305056.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[:, df.columns.str.contains("|".join(merge_keys)) | df.columns.str.contains("(min|max|median|mode|avg)", case=False)]


In [69]:
# Upload refactor_base_cohort to BigQuery as a temporary table
table_id_final_base_vitals = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_vitals"

# Upload the DataFrame to BigQuery
final_base_vitals.to_gbq(
    destination_table=table_id_final_base_vitals,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_vitals to {table_id_final_base_vitals}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/1786291503.py:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_vitals.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 2174.34it/s]

Uploaded final_base_vitals to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_vitals


In [70]:
final_base_vitals.describe()

,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,temp_min_c,temp_avg_c,temp_max_c,temp_median_c,temp_mode_c,resp_min,resp_avg,resp_max,resp_median,resp_mode,hr_min,hr_avg,hr_max,hr_median,hr_mode,sysbp_min,sysbp_avg,sysbp_max,sysbp_median,sysbp_mode,diabp_min,diabp_avg,diabp_max,diabp_median,diabp_mode,spo2_min,spo2_avg,spo2_max,spo2_median,spo2_mode
count,198015.0,198015.0,198015,193967.000000,193967.000000,193967.000000,193967.000000,193967.000000,197270.000000,197270.000000,197270.000000,197270.000000,197270.000000,197692.000000,197692.000000,197692.000000,197692.000000,197692.000000,197531.000000,197531.000000,197531.000000,197531.000000,197531.000000,197507.000000,197507.000000,197507.000000,197507.000000,197507.000000,197007.000000,197007.000000,197007.000000,197007.000000,197007.000000
mean,131266428820.500565,698515346.39747,2020-02-19 05:25:18.269525,36.912416,37.048301,37.192325,36.968157,36.931954,17.498560,19.727260,22.218259,19.163350,18.533147,85.532829,93.110602,101.242458,91.464419,88.673482,118.076894,127.452507,137.324131,125.031382,120.670411,66.818801,73.895630,81.918362,71.809232,68.901138,94.553462,96.770963,98.272168,96.766770,96.321131
min,131012566723.0,380223319.0,2011-02-13 21:51:00,27.000000,27.166667,27.166667,27.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,11.000000,0.000000,0.000000,0.000000,46.666667,52.000000,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000
25%,131236541158.0,531432731.5,2017-07-19 19:28:00,36.500000,36.611111,36.666667,36.555556,36.500000,16.000000,17.400000,18.000000,17.000000,16.000000,71.000000,79.000000,86.000000,77.000000,74.000000,103.000000,112.428571,120.000000,110.000000,105.000000,56.000000,64.187500,70.000000,62.000000,59.000000,94.000000,95.666667,97.000000,95.000000,95.000000
50%,131297689843.0,690452176.0,2020-11-03 00:16:00,36.777778,36.888889,37.000000,36.833333,36.800000,18.000000,18.666667,20.000000,18.000000,18.000000,85.000000,92.000000,100.000000,90.000000,88.000000,117.000000,126.000000,135.000000,123.000000,119.000000,66.000000,73.200000,80.000000,71.000000,68.000000,96.000000,97.333333,99.000000,97.000000,97.000000
75%,131352357929.0,868315018.5,2023-04-10 02:57:00,37.166667,37.388889,37.555556,37.222222,37.200000,19.000000,21.000000,24.000000,20.000000,20.000000,99.000000,106.000000,115.000000,104.000000,102.000000,133.000000,141.000000,152.000000,139.000000,135.000000,77.000000,82.692308,92.000000,81.000000,79.000000,98.000000,98.750000,100.000000,99.000000,99.000000
max,131398322480.0,1033262602.0,2025-03-28 23:18:00,42.055556,42.055556,42.222222,42.055556,42.200000,90.000000,90.000000,90.000000,90.000000,90.000000,196.000000,208.083333,250.000000,250.000000,250.000000,242.000000,243.500000,250.000000,242.000000,246.000000,150.000000,150.000000,150.000000,150.000000,150.000000,100.000000,100.000000,100.000000,100.000000,100.000000
std,113226713.373962,188677836.550717,NaN,0.833950,0.818988,0.907342,0.817489,0.825060,3.798407,3.961642,6.025172,3.972985,3.898559,20.780804,19.280336,22.305891,19.668376,20.095873,23.612859,21.750924,24.157563,22.243434,23.135450,15.544206,13.960799,16.951034,14.406042,15.224147,8.451265,3.634775,2.174069,3.428317,5.452097


# Complete blood count with differntial CBCd
### WBC, HgB, PLT, ANC, ALC, % Neutrophils, %Lymphocytes, glucose, lactate, crp, procalcitonin

##### treat crp and high sensitivity crp differnlty 

In [147]:
# V2 refactor code
query = f"""
with all_lab_result as (
    select anon_id,
    pat_enc_csn_id_coded,
    lab_name,
    base_name,
    ord_value,
    reference_unit,
    order_time_jittered,
    from `som-nero-phi-jonc101.shc_core_2024.lab_result`
    where anon_id in (select distinct(anon_id) from {table_id_final_base_bmi_age_race_gender_adi_temp}) 
    
),
labs AS (
    SELECT 
        c.anon_id,
       c.pat_enc_csn_id_coded,
       c.order_proc_id_coded,
        CASE 
            WHEN (LOWER(lr.base_name) = 'wbc'
                    AND LOWER(lr.reference_unit) IN ('thousand/ul','k/ul','10x3/ul','10*3/ul','x10e3/ul')) THEN SAFE_CAST(lr.ord_value AS FLOAT64)
            WHEN LOWER(lr.base_name) = 'wbc'
                    AND lr.reference_unit = '/uL' 
                        THEN SAFE_CAST(lr.ord_value AS FLOAT64)/1000
            END AS wbc,
        CASE 
        WHEN LOWER(lr.lab_name) LIKE '%neutrophils%'
        # WHEN LOWER(lr.lab_name)LIKE '%neutrophils%' 
        # or LOWER (lr.lab_name) LIKE '%neut,abs%' or
        # LOWER (lr.lab_name) LIKE '%neutrophil,absolute%' or
        # LOWER (lr.lab_name) LIKE '%neut,abs(seg+band)%' or
        # LOWER (lr.lab_name) LIKE '%neutrophils absolute%' or
        # LOWER (lr.lab_name) LIKE '%neuts,abs%' or
        # LOWER (lr.lab_name) LIKE '%absolute neutrophile%'
        AND lr.reference_unit = '%' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        END AS neutrophils,
        CASE 
            WHEN LOWER(lr.lab_name) LIKE '%lymphocytes%' AND lr.reference_unit = '%' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        END AS lymphocytes,
        -- CASE WHEN LOWER(lr.base_name) LIKE '%bands%' THEN SAFE_CAST(lr.ord_value AS FLOAT64) END AS bands,
        CASE 
            WHEN LOWER(lr.base_name) = 'hgb' AND LOWER(trim(lr.reference_unit)) = 'mg/dl' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
            WHEN LOWER(lr.base_name) = 'hgb' AND LOWER(trim(lr.reference_unit)) like '%g/dl%' or LOWER(trim(lr.reference_unit)) like '%gm/dl%' THEN SAFE_CAST(lr.ord_value AS FLOAT64) * 1000
        END AS hgb,
        CASE 
            WHEN LOWER(lr.base_name) = 'plt' AND LOWER(lr.reference_unit) IN ('x10e3/ul','10x3/ul','k/ul','10*3/ul','thousand/ul') THEN SAFE_CAST(lr.ord_value AS FLOAT64)
            WHEN LOWER(lr.base_name) = 'plt' AND LOWER(lr.reference_unit) = 'ul' THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 1000
        END AS plt,
        # CASE 
        #     WHEN LOWER(lr.base_name) = 'na' AND LOWER(lr.reference_unit) = 'mmol/l' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        # END AS na,
        # CASE 
        #     WHEN (LOWER(lr.base_name) = 'hco3' AND LOWER(lr.reference_unit) like any ('meq/l','mmol/l')) THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        #     WHEN (LOWER(lr.base_name) = 'co2' AND LOWER(lr.reference_unit) like any ('meq/l','mmol/l')) THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        # END AS hco3,      
        # CASE 
        #     WHEN LOWER(lr.base_name) = 'bun' AND LOWER(lr.reference_unit) = 'mg/dl' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        # END AS bun,
        # CASE 
        #     WHEN LOWER(lr.base_name) = 'cr' AND LOWER(lr.reference_unit) = 'mg/dl' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        # END AS cr,
        CASE WHEN LOWER(lr.base_name) LIKE 'glucose' THEN SAFE_CAST(lr.ord_value AS FLOAT64) END AS glucose,
        CASE 
            WHEN LOWER(lr.base_name) = 'lac' AND LOWER(lr.reference_unit) IN ('mmol/l', 'mmole/l') THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        END AS lactate,               
        CASE 
            WHEN LOWER(lr.base_name) like '%crp%' AND not LOWER(lr.base_name) like '%h%'AND LOWER(lr.reference_unit) = 'mg/dl' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
            WHEN LOWER(lr.base_name) like '%crp%' AND not LOWER(lr.base_name) like '%h%'AND LOWER(lr.reference_unit) = 'mg/l' THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 10
        END AS crp,
         CASE 
            WHEN LOWER(lr.base_name) like '%crp%' AND LOWER(lr.base_name) like '%h%'  AND LOWER(lr.reference_unit) = 'mg/dl' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
            WHEN LOWER(lr.base_name) like '%crp%' AND LOWER(lr.base_name) like '%h%'  AND LOWER(lr.reference_unit) = 'mg/l' THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 10
        END AS crp_high_sensitivity,

        CASE 
            WHEN LOWER(lr.lab_name) LIKE 'procalcitonin' AND LOWER(lr.reference_unit) = 'ng/ml' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
        END AS procalcitonin
  
    FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
    LEFT JOIN all_lab_result lr 
    USING (anon_id, pat_enc_csn_id_coded)
    WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, Day) BETWEEN -2 AND 0
),
lab_records as 
(
SELECT 
    anon_id,
    pat_enc_csn_id_coded,
    order_proc_id_coded,
    
    ROUND(MIN(wbc), 2) AS min_wbc,
    ROUND(MAX(wbc), 2) AS max_wbc,
    ROUND(AVG(wbc), 2) AS avg_wbc,
    ROUND(APPROX_QUANTILES(wbc, 100)[OFFSET(50)], 2) AS median_wbc,
    
    ROUND(MIN(neutrophils), 2) AS min_neutrophils,
    ROUND(MAX(neutrophils), 2) AS max_neutrophils,
    ROUND(AVG(neutrophils), 2) AS avg_neutrophils,
    ROUND(APPROX_QUANTILES(neutrophils, 100)[OFFSET(50)], 2) AS median_neutrophils,
    
    ROUND(MIN(lymphocytes), 2) AS min_lymphocytes,
    ROUND(MAX(lymphocytes), 2) AS max_lymphocytes,
    ROUND(AVG(lymphocytes), 2) AS avg_lymphocytes,
    ROUND(APPROX_QUANTILES(lymphocytes, 100)[OFFSET(50)], 2) AS median_lymphocytes,
    
    
    ROUND(MIN(hgb), 2) AS min_hgb,
    ROUND(MAX(hgb), 2) AS max_hgb,
    ROUND(AVG(hgb), 2) AS avg_hgb,
    ROUND(APPROX_QUANTILES(hgb, 100)[OFFSET(50)], 2) AS median_hgb,

    ROUND(MIN(plt), 2) AS min_plt,
    ROUND(MAX(plt), 2) AS max_plt,
    ROUND(AVG(plt), 2) AS avg_plt,
    ROUND(APPROX_QUANTILES(plt, 100)[OFFSET(50)], 2) AS median_plt,

    # ROUND(MIN(na), 2) AS min_na,
    # ROUND(MAX(na), 2) AS max_na,
    # ROUND(AVG(na), 2) AS avg_na,
    # ROUND(APPROX_QUANTILES(na, 100)[OFFSET(50)], 2) AS median_na,

    # ROUND(MIN(hco3), 2) AS min_hco3,
    # ROUND(MAX(hco3), 2) AS max_hco3,
    # ROUND(AVG(hco3), 2) AS avg_hco3,
    # ROUND(APPROX_QUANTILES(hco3, 100)[OFFSET(50)], 2) AS median_hco3,

    # ROUND(MIN(bun), 2) AS min_bun,
    # ROUND(MAX(bun), 2) AS max_bun,
    # ROUND(AVG(bun), 2) AS avg_bun,
    # ROUND(APPROX_QUANTILES(bun, 100)[OFFSET(50)], 2) AS median_bun,

    # ROUND(MIN(cr), 2) AS min_cr,
    # ROUND(MAX(cr), 2) AS max_cr,
    # ROUND(AVG(cr), 2) AS avg_cr,
    # ROUND(APPROX_QUANTILES(cr, 100)[OFFSET(50)], 2) AS median_cr,

    ROUND(MIN(glucose), 2) AS min_glucose,
    ROUND(MAX(glucose), 2) AS max_glucose,
    ROUND(AVG(glucose), 2) AS avg_glucose,
    ROUND(APPROX_QUANTILES(glucose, 100)[OFFSET(50)], 2) AS median_glucose,

    ROUND(MIN(lactate), 2) AS min_lactate,
    ROUND(MAX(lactate), 2) AS max_lactate,
    ROUND(AVG(lactate), 2) AS avg_lactate,
    ROUND(APPROX_QUANTILES(lactate, 100)[OFFSET(50)], 2) AS median_lactate,

    ROUND(MIN(procalcitonin), 2) AS min_procalcitonin,
    ROUND(MAX(procalcitonin), 2) AS max_procalcitonin,
    ROUND(AVG(procalcitonin), 2) AS avg_procalcitonin,
    ROUND(APPROX_QUANTILES(procalcitonin, 100)[OFFSET(50)], 2) AS median_procalcitonin,

    ROUND(MIN(crp), 2) AS min_crp,
    ROUND(MAX(crp), 2) AS max_crp,
    ROUND(AVG(crp), 2) AS avg_crp,
    ROUND(APPROX_QUANTILES(crp, 100)[OFFSET(50)], 2) AS median_crp,

    ROUND(MIN(crp_high_sensitivity), 2) AS min_crp_high_sensitivity,
    ROUND(MAX(crp_high_sensitivity), 2) AS max_crp_high_sensitivity,
    ROUND(AVG(crp_high_sensitivity), 2) AS avg_crp_high_sensitivity,
    ROUND(APPROX_QUANTILES(crp_high_sensitivity, 100)[OFFSET(50)], 2) AS median_crp_high_sensitivity,


FROM labs
GROUP BY 
    anon_id,
    pat_enc_csn_id_coded,
    order_proc_id_coded
)   
select c.*,
min_wbc,
max_wbc,
avg_wbc,
median_wbc,
min_neutrophils,
max_neutrophils,
avg_neutrophils,
median_neutrophils,
min_lymphocytes,
max_lymphocytes,
avg_lymphocytes,
median_lymphocytes,
min_hgb,
max_hgb,
avg_hgb,
median_hgb,
min_plt,
max_plt,
avg_plt,
median_plt,
# min_na,
# max_na,
# avg_na,
# median_na,
# min_hco3,
# max_hco3,
# avg_hco3,
# median_hco3,
# min_bun,
# max_bun,
# avg_bun,
# median_bun,
# min_cr,
# max_cr,
# avg_cr,
# median_cr,
min_lactate,
max_lactate,
avg_lactate,
median_lactate,
min_glucose,
max_glucose,
avg_glucose,
median_glucose,
min_procalcitonin,
max_procalcitonin,
avg_procalcitonin,
median_procalcitonin,
min_crp,
max_crp,
avg_crp,
median_crp,
min_crp_high_sensitivity,
max_crp_high_sensitivity,
avg_crp_high_sensitivity,
median_crp_high_sensitivity
from {table_id_final_base_bmi_age_race_gender_adi_temp} c
left join lab_records using(anon_id,pat_enc_csn_id_coded,order_proc_id_coded)
"""

final_base_labs_no_anc_or_alc = client.query(query).to_dataframe()

## WBC


In [71]:
query_wbc = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
wbc_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(lr.base_name) = 'wbc' AND LOWER(lr.reference_unit) IN ('thousand/ul','k/ul','10x3/ul','10*3/ul','x10e3/ul')
        THEN SAFE_CAST(lr.ord_value AS FLOAT64)
      WHEN LOWER(lr.base_name) = 'wbc' AND LOWER(lr.reference_unit) IN ('/ul','ul')
        THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 1000
    END AS wbc
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
wbc_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(wbc),2) AS min_wbc,
    ROUND(MAX(wbc),2) AS max_wbc,
    ROUND(AVG(wbc),2) AS avg_wbc,
    ROUND(APPROX_QUANTILES(wbc,100)[OFFSET(50)],2) AS median_wbc
  FROM wbc_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_wbc, a.max_wbc, a.avg_wbc, a.median_wbc
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN wbc_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_wbc_only = client.query(query_wbc).to_dataframe()


### Neutrophils (%)

In [74]:

query_neut = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),

neut_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(TRIM(lr.lab_name)) IN ('neutrophil %')
      THEN SAFE_CAST(lr.ord_value AS FLOAT64)
    END AS neutrophils
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
neut_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(neutrophils),2) AS min_neutrophils,
    ROUND(MAX(neutrophils),2) AS max_neutrophils,
    ROUND(AVG(neutrophils),2) AS avg_neutrophils,
    ROUND(APPROX_QUANTILES(neutrophils,100)[OFFSET(50)],2) AS median_neutrophils
  FROM neut_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_neutrophils, a.max_neutrophils, a.avg_neutrophils, a.median_neutrophils
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN neut_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_neutrophils_only = client.query(query_neut).to_dataframe()


In [75]:
final_base_neutrophils_only.isnull().mean()

anon_id                             0.000000
pat_enc_csn_id_coded                0.000000
order_proc_id_coded                 0.000000
blood_culture_order_datetime_utc    0.000000
min_neutrophils                     0.269323
max_neutrophils                     0.269323
avg_neutrophils                     0.269323
median_neutrophils                  0.269323
dtype: float64

#### ANC

In [76]:
# unit is K/uL
query_anc = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),

neut_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE 
      WHEN trim(lr.base_name) = 'NEUTAB'
      THEN SAFE_CAST(lr.ord_value AS FLOAT64)
    END AS neutrophils
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
neut_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(neutrophils),2) AS min_anc,
    ROUND(MAX(neutrophils),2) AS max_anc,
    ROUND(AVG(neutrophils),2) AS avg_anc,
    ROUND(APPROX_QUANTILES(neutrophils,100)[OFFSET(50)],2) AS median_anc
  FROM neut_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_anc, a.max_anc, a.avg_anc, a.median_anc
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN neut_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_anc_only = client.query(query_anc).to_dataframe()


In [78]:
final_base_anc_only.describe()

,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_anc,max_anc,avg_anc,median_anc
count,198015.0,198015.0,198015,175839.000000,175839.000000,175839.000000,175839.000000
mean,131266428820.500565,698515346.39747,2020-02-19 05:25:18.269525,8.539111,8.824949,8.681601,8.554936
min,131012566723.0,380223319.0,2011-02-13 21:51:00,0.000000,0.000000,0.000000,0.000000
25%,131236541158.0,531432731.5,2017-07-19 19:28:00,4.750000,4.910000,4.850000,4.760000
50%,131297689843.0,690452176.0,2020-11-03 00:16:00,7.340000,7.590000,7.490000,7.360000
75%,131352357929.0,868315018.5,2023-04-10 02:57:00,11.050000,11.410000,11.220000,11.070000
max,131398322480.0,1033262602.0,2025-03-28 23:18:00,199.000000,219.300000,209.150000,199.000000
std,113226713.373962,188677836.550717,NaN,5.616241,5.818828,5.684781,5.628527


In [ ]:
# to-do, check the range of anc with Nick

### lymphocytes (%)

In [79]:

query_lymph = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
lymph_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN trim(lr.base_name) in ('LYM')
      THEN SAFE_CAST(lr.ord_value AS FLOAT64)
    END AS lymphocytes
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
lymph_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(lymphocytes),2) AS min_lymphocytes,
    ROUND(MAX(lymphocytes),2) AS max_lymphocytes,
    ROUND(AVG(lymphocytes),2) AS avg_lymphocytes,
    ROUND(APPROX_QUANTILES(lymphocytes,100)[OFFSET(50)],2) AS median_lymphocytes
  FROM lymph_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_lymphocytes, a.max_lymphocytes, a.avg_lymphocytes, a.median_lymphocytes
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN lymph_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_lymphocytes_only = client.query(query_lymph).to_dataframe()


In [81]:
final_base_lymphocytes_only.describe()

,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_lymphocytes,max_lymphocytes,avg_lymphocytes,median_lymphocytes
count,198015.0,198015.0,198015,191396.000000,191396.000000,191396.000000,191396.000000
mean,131266428820.500565,698515346.39747,2020-02-19 05:25:18.269525,14.140034,14.816643,14.474589,14.172517
min,131012566723.0,380223319.0,2011-02-13 21:51:00,0.000000,0.000000,0.000000,0.000000
25%,131236541158.0,531432731.5,2017-07-19 19:28:00,6.100000,6.500000,6.400000,6.100000
50%,131297689843.0,690452176.0,2020-11-03 00:16:00,11.000000,11.700000,11.400000,11.000000
75%,131352357929.0,868315018.5,2023-04-10 02:57:00,18.600000,19.500000,19.000000,18.700000
max,131398322480.0,1033262602.0,2025-03-28 23:18:00,100.000000,100.000000,100.000000,100.000000
std,113226713.373962,188677836.550717,NaN,11.943616,12.218144,11.993386,11.952841


### ALC

In [82]:
# convert unit to k/ul
query_alc = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
lymph_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN trim(lr.base_name) in ('LYMAB') AND  LOWER(trim(lr.reference_unit)) = 'k/ul' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
      WHEN trim(lr.base_name) in ('LYMAB') AND  LOWER(trim(lr.reference_unit)) = 'ul' THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 1000
      WHEN trim(lr.base_name) in ('LYMAB') AND  LOWER(trim(lr.reference_unit)) = 'cells/uL' THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 1000
      WHEN trim(lr.base_name) in ('LYMAB') AND  LOWER(trim(lr.reference_unit)) is null THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 1000
      ELSE NULL
    END AS lymphocytes
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
lymph_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(lymphocytes),2) AS min_alc,
    ROUND(MAX(lymphocytes),2) AS max_alc,
    ROUND(AVG(lymphocytes),2) AS avg_alc,
    ROUND(APPROX_QUANTILES(lymphocytes,100)[OFFSET(50)],2) AS median_alc
  FROM lymph_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_alc, a.max_alc, a.avg_alc, a.median_alc
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN lymph_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_alc_only = client.query(query_alc).to_dataframe()


### hgb

In [83]:
query_hgb = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
hgb_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(lr.base_name) = 'hgb' AND LOWER(TRIM(lr.reference_unit)) = 'mg/dl' 
        THEN SAFE_CAST(lr.ord_value AS FLOAT64) /1000
      WHEN LOWER(lr.base_name) = 'hgb' AND (LOWER(TRIM(lr.reference_unit)) LIKE '%g/dl%' OR LOWER(TRIM(lr.reference_unit)) LIKE '%gm/dl%')
        THEN SAFE_CAST(lr.ord_value AS FLOAT64) 
    END AS hgb_mgdl
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
hgb_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(hgb_mgdl),2) AS min_hgb,
    ROUND(MAX(hgb_mgdl),2) AS max_hgb,
    ROUND(AVG(hgb_mgdl),2) AS avg_hgb,
    ROUND(APPROX_QUANTILES(hgb_mgdl,100)[OFFSET(50)],2) AS median_hgb
  FROM hgb_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_hgb, a.max_hgb, a.avg_hgb, a.median_hgb
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN hgb_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_hgb_only = client.query(query_hgb).to_dataframe()


In [84]:
final_base_hgb_only.isnull().mean()

anon_id                             0.000000
pat_enc_csn_id_coded                0.000000
order_proc_id_coded                 0.000000
blood_culture_order_datetime_utc    0.000000
min_hgb                             0.007681
max_hgb                             0.007681
avg_hgb                             0.007681
median_hgb                          0.007681
dtype: float64

In [166]:
final_base_hgb_only.describe()

,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_hgb,max_hgb,avg_hgb,median_hgb
count,189319.0,189319.0,189319,188366.000000,188366.000000,188366.000000,188366.000000
mean,131263689432.992676,694581778.29726,2020-01-19 23:11:32.679446,11.080912,12.010837,11.527763,11.291654
min,131012566723.0,380223319.0,2011-02-13 21:51:00,0.000000,2.200000,2.200000,0.000000
25%,131229508630.0,525445444.5,2017-05-12 10:43:30,9.400000,10.300000,9.850000,9.600000
50%,131295784833.0,684900639.0,2020-09-29 22:38:00,11.200000,12.100000,11.600000,11.300000
75%,131351487545.0,865110719.5,2023-03-27 22:33:00,12.800000,13.600000,13.170000,12.900000
max,131398320569.0,1033259720.0,2025-03-28 23:18:00,25.000000,25.000000,25.000000,25.000000
std,114829740.882937,189243033.786885,NaN,2.412705,2.303582,2.277948,2.310738


### platelets


In [85]:
# unit is k/ul
query_plt = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
plt_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(lr.base_name) = 'plt' AND LOWER(lr.reference_unit) IN ('x10e3/ul','10x3/ul','10*3/ul','k/ul','thousand/ul')
        THEN SAFE_CAST(lr.ord_value AS FLOAT64) 
      WHEN LOWER(lr.base_name) = 'plt' AND LOWER(lr.reference_unit) IN ('/ul','ul')
        THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 1000
    END AS plt_kul
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
plt_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(plt_kul),2) AS min_plt,
    ROUND(MAX(plt_kul),2) AS max_plt,
    ROUND(AVG(plt_kul),2) AS avg_plt,
    ROUND(APPROX_QUANTILES(plt_kul,100)[OFFSET(50)],2) AS median_plt
  FROM plt_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_plt, a.max_plt, a.avg_plt, a.median_plt
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN plt_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_plt_only = client.query(query_plt).to_dataframe()


### glucose

In [86]:
# unit is mg/dl
query_glucose = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
glu_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE WHEN LOWER(lr.base_name) = 'glu' THEN SAFE_CAST(lr.ord_value AS FLOAT64) END AS glucose
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
glu_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(glucose),2) AS min_glucose,
    ROUND(MAX(glucose),2) AS max_glucose,
    ROUND(AVG(glucose),2) AS avg_glucose,
    ROUND(APPROX_QUANTILES(glucose,100)[OFFSET(50)],2) AS median_glucose
  FROM glu_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_glucose, a.max_glucose, a.avg_glucose, a.median_glucose
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN glu_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_glucose_only = client.query(query_glucose).to_dataframe()


In [179]:
final_base_glucose_only.describe()

,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_glucose,max_glucose,avg_glucose,median_glucose
count,189319.0,189319.0,189319,187676.000000,187676.000000,187676.000000,187676.000000
mean,131263689432.992676,694581778.29726,2020-01-19 23:11:32.679446,124.662072,154.291588,138.592062,132.418397
min,131012566723.0,380223319.0,2011-02-13 21:51:00,3.000000,31.000000,31.000000,13.000000
25%,131229508630.0,525445444.5,2017-05-12 10:43:30,96.000000,106.000000,102.500000,98.000000
50%,131295784833.0,684900639.0,2020-09-29 22:38:00,111.000000,126.000000,119.500000,114.000000
75%,131351487545.0,865110719.5,2023-03-27 22:33:00,136.000000,167.000000,150.000000,143.000000
max,131398320569.0,1033259720.0,2025-03-28 23:18:00,1136.000000,2146.000000,1214.430000,1188.000000
std,114829740.882937,189243033.786885,NaN,50.754564,90.826881,62.912209,60.858318


### lactate


In [87]:
# unit is mmol/l
query_lactate = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
lac_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(lr.base_name) in ('lac', 'lacpoc', 'lacwbl') AND (LOWER(lr.reference_unit) IN ('mmol/l','mmole/l','mmoll') 
        or lr.reference_unit is null)
        THEN SAFE_CAST(lr.ord_value AS FLOAT64)
      WHEN LOWER(lr.base_name) in ('lac', 'lacpoc', 'lacwbl') AND LOWER(lr.reference_unit) IN ('mg/dl')
        THEN SAFE_CAST(lr.ord_value AS FLOAT64) /9.008
    END AS lactate
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
lac_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(lactate),2) AS min_lactate,
    ROUND(MAX(lactate),2) AS max_lactate,
    ROUND(AVG(lactate),2) AS avg_lactate,
    ROUND(APPROX_QUANTILES(lactate,100)[OFFSET(50)],2) AS median_lactate
  FROM lac_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_lactate, a.max_lactate, a.avg_lactate, a.median_lactate
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN lac_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_lactate_only = client.query(query_lactate).to_dataframe()


In [198]:
final_base_lactate_only.describe()

,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_lactate,max_lactate,avg_lactate,median_lactate
count,189319.0,189319.0,189319,146872.000000,146872.000000,146872.000000,146872.000000
mean,131263689432.992676,694581778.29726,2020-01-19 23:11:32.679446,1.481980,2.031742,1.740301,1.609420
min,131012566723.0,380223319.0,2011-02-13 21:51:00,0.200000,0.300000,0.300000,0.200000
25%,131229508630.0,525445444.5,2017-05-12 10:43:30,0.900000,1.000000,0.980000,0.920000
50%,131295784833.0,684900639.0,2020-09-29 22:38:00,1.300000,1.500000,1.420000,1.320000
75%,131351487545.0,865110719.5,2023-03-27 22:33:00,1.770000,2.320000,2.000000,1.860000
max,131398320569.0,1033259720.0,2025-03-28 23:18:00,20.400000,28.200000,21.400000,20.400000
std,114829740.882937,189243033.786885,NaN,1.062813,1.901996,1.356666,1.267324


### regular crp: continue from here:

In [34]:
query_crp = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
crp_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(lr.base_name) LIKE '%crp%' AND NOT LOWER(lr.base_name) LIKE '%h%'
           AND LOWER(lr.reference_unit) = 'mg/dl'
        THEN SAFE_CAST(lr.ord_value AS FLOAT64)
      WHEN LOWER(lr.base_name) LIKE '%crp%' AND NOT LOWER(lr.base_name) LIKE '%h%'
           AND LOWER(lr.reference_unit) = 'mg/l'
        THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 10
    END AS crp_mgdl
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
crp_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(crp_mgdl),2) AS min_crp,
    ROUND(MAX(crp_mgdl),2) AS max_crp,
    ROUND(AVG(crp_mgdl),2) AS avg_crp,
    ROUND(APPROX_QUANTILES(crp_mgdl,100)[OFFSET(50)],2) AS median_crp
  FROM crp_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_crp, a.max_crp, a.avg_crp, a.median_crp
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN crp_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_crp_only = client.query(query_crp).to_dataframe()


In [35]:
final_base_crp_only.isnull().mean()

anon_id                             0.000000
pat_enc_csn_id_coded                0.000000
order_proc_id_coded                 0.000000
blood_culture_order_datetime_utc    0.000000
min_crp                             0.865993
max_crp                             0.865993
avg_crp                             0.865993
median_crp                          0.865993
dtype: float64

In [36]:
### crp high sensitivity
query_crp_hs = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
crphs_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(lr.base_name) LIKE '%crp%' AND LOWER(lr.base_name) LIKE '%h%'
           AND LOWER(lr.reference_unit) = 'mg/dl'
        THEN SAFE_CAST(lr.ord_value AS FLOAT64)
      WHEN LOWER(lr.base_name) LIKE '%crp%' AND LOWER(lr.base_name) LIKE '%h%'
           AND LOWER(lr.reference_unit) = 'mg/l'
        THEN SAFE_CAST(lr.ord_value AS FLOAT64) / 10
    END AS crp_hs_mgdl
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
crphs_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(crp_hs_mgdl),2) AS min_crp_high_sensitivity,
    ROUND(MAX(crp_hs_mgdl),2) AS max_crp_high_sensitivity,
    ROUND(AVG(crp_hs_mgdl),2) AS avg_crp_high_sensitivity,
    ROUND(APPROX_QUANTILES(crp_hs_mgdl,100)[OFFSET(50)],2) AS median_crp_high_sensitivity
  FROM crphs_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_crp_high_sensitivity, a.max_crp_high_sensitivity, a.avg_crp_high_sensitivity, a.median_crp_high_sensitivity
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN crphs_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_crp_hs_only = client.query(query_crp_hs).to_dataframe()


In [37]:
final_base_crp_hs_only.isnull().mean()

anon_id                             0.000000
pat_enc_csn_id_coded                0.000000
order_proc_id_coded                 0.000000
blood_culture_order_datetime_utc    0.000000
min_crp_high_sensitivity            0.999187
max_crp_high_sensitivity            0.999187
avg_crp_high_sensitivity            0.999187
median_crp_high_sensitivity         0.999187
dtype: float64

### procalcitonin


In [38]:
query_pct = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
pct_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(TRIM(lr.lab_name)) = 'procalcitonin' 
    #   AND LOWER(TRIM(lr.reference_unit)) = 'ng/ml'
        THEN SAFE_CAST(lr.ord_value AS FLOAT64)
    END AS procalcitonin
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, DAY) BETWEEN -2 AND 0
),
pct_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(procalcitonin),2) AS min_procalcitonin,
    ROUND(MAX(procalcitonin),2) AS max_procalcitonin,
    ROUND(AVG(procalcitonin),2) AS avg_procalcitonin,
    ROUND(APPROX_QUANTILES(procalcitonin,100)[OFFSET(50)],2) AS median_procalcitonin
  FROM pct_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_procalcitonin, a.max_procalcitonin, a.avg_procalcitonin, a.median_procalcitonin
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN pct_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_pct_only = client.query(query_pct).to_dataframe()


In [ ]:
# query_pct_outliers = f"""
# WITH all_lab_result AS (
#   SELECT 
#     anon_id, 
#     pat_enc_csn_id_coded, 
#     lab_name, 
#     base_name, 
#     ord_value, 
#     reference_unit, 
#     order_time_jittered
#   FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
#   WHERE anon_id IN (
#     SELECT DISTINCT anon_id 
#     FROM {table_id_final_base_bmi_age_race_gender_adi_temp}
#   )
# ),

# lac_raw AS (
#   SELECT
#     c.anon_id,
#     c.pat_enc_csn_id_coded,
#     c.order_proc_id_coded,
#     c.blood_culture_order_datetime_utc,
#     lr.lab_name,
#     lr.base_name,
#     lr.ord_value,
#     lr.reference_unit,
#     lr.order_time_jittered,
#   CASE
#         WHEN LOWER(TRIM(lr.lab_name)) = 'procalcitonin' 
#       #   AND LOWER(TRIM(lr.reference_unit)) = 'ng/ml'
#           THEN SAFE_CAST(lr.ord_value AS FLOAT64)
#       END AS procalcitonin
#   FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
#   LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
#   WHERE TIMESTAMP_DIFF(
#           lr.order_time_jittered,
#           c.blood_culture_order_datetime_utc,
#           HOUR
#         ) BETWEEN -48 AND 0
# )

# SELECT *
# FROM lac_raw
# WHERE procalcitonin IS NOT NULL
#   AND (procalcitonin > 10 OR procalcitonin < 0)  -- flag possible outliers
# ORDER BY procalcitonin DESC
# """

# # Run and collect into a DataFrame
# pct_outliers_df = client.query(query_pct_outliers).to_dataframe()

# # Display or inspect
# print(f"Found {len(pct_outliers_df)} potential lactate outliers")
# display(pct_outliers_df.head(20))


Found 1092 potential lactate outliers


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,lab_name,base_name,ord_value,reference_unit,order_time_jittered,procalcitonin
0,JC1263713,131296347305,686361698,2020-11-07 07:03:00,Procalcitonin,PROCTL,360.40,ng/mL,2020-11-07 02:02:00,360.40
1,JC1263713,131296347305,686361699,2020-11-07 07:03:00,Procalcitonin,PROCTL,360.40,ng/mL,2020-11-07 02:02:00,360.40
2,JC1481812,131214894433,515741935,2017-02-08 20:01:00,Procalcitonin,PROCTL,359.0,ng/mL,2017-02-08 14:39:00,359.00
3,JC1481812,131214894433,515741936,2017-02-08 20:01:00,Procalcitonin,PROCTL,359.0,ng/mL,2017-02-08 14:39:00,359.00
4,JC884827,131385912470,987355272,2024-09-12 16:11:00,Procalcitonin,PROCTL,344.40,ng/mL,2024-09-12 10:20:00,344.40
5,JC884827,131385912470,987355270,2024-09-12 16:11:00,Procalcitonin,PROCTL,344.40,ng/mL,2024-09-12 10:20:00,344.40
6,JC1851524,131260739912,585731298,2018-11-20 02:25:00,Procalcitonin,PROCTL,338.08,ng/mL,2018-11-20 01:54:00,338.08
7,JC1851524,131260739912,585731297,2018-11-20 02:25:00,Procalcitonin,PROCTL,338.08,ng/mL,2018-11-20 01:54:00,338.08
8,JC6257500,131380418743,968697866,2024-06-12 06:28:00,Procalcitonin,PROCTL,329.20,ng/mL,2024-06-12 06:53:00,329.20
9,JC6257500,131380418743,968697867,2024-06-12 06:28:00,Procalcitonin,PROCTL,329.20,ng/mL,2024-06-12 06:53:00,329.20


In [40]:
final_base_pct_only.describe()

,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_procalcitonin,max_procalcitonin,avg_procalcitonin,median_procalcitonin
count,189319.0,189319.0,189319,29452.000000,29452.000000,29452.000000,29452.000000
mean,131263689432.992676,694581778.29726,2020-01-19 23:11:32.679446,2.770384,2.950443,2.860879,2.779049
min,131012566723.0,380223319.0,2011-02-13 21:51:00,0.020000,0.020000,0.020000,0.020000
25%,131229508630.0,525445444.5,2017-05-12 10:43:30,0.100000,0.100000,0.100000,0.100000
50%,131295784833.0,684900639.0,2020-09-29 22:38:00,0.230000,0.230000,0.230000,0.230000
75%,131351487545.0,865110719.5,2023-03-27 22:33:00,0.800000,0.840000,0.820000,0.800000
max,131398320569.0,1033259720.0,2025-03-28 23:18:00,671.750000,671.750000,671.750000,671.750000
std,114829740.882937,189243033.786885,NaN,15.529139,16.120462,15.750403,15.547620


In [220]:
final_base_pct_only.isnull().mean()

anon_id                             0.000000
pat_enc_csn_id_coded                0.000000
order_proc_id_coded                 0.000000
blood_culture_order_datetime_utc    0.000000
min_procalcitonin                   0.844432
max_procalcitonin                   0.844432
avg_procalcitonin                   0.844432
median_procalcitonin                0.844432
dtype: float64

# Inflamatory markers
## procalcitonin, crp, done in lab section

# Blood gas
## glucose and lactate , done in lab section

In [88]:
import pandas as pd
from functools import reduce

# List of all per-vital DataFrames
labs_df = [final_base_wbc_only, final_base_neutrophils_only,final_base_anc_only, final_base_lymphocytes_only, final_base_alc_only, final_base_hgb_only, final_base_plt_only, final_base_glucose_only, final_base_lactate_only]

# Merge keys
merge_keys = ["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "blood_culture_order_datetime_utc"]

# Keep only key columns + aggregate metrics (min, max, median, mode)
dfs_clean = [
    df.loc[:, df.columns.str.contains("|".join(merge_keys)) | df.columns.str.contains("(min|max|median|mode|avg)", case=False)]
    for df in labs_df
]

# Merge all DataFrames on the same keys
final_base_labs_without_cpr_or_pct = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how="outer"), dfs_clean)


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/1548309106.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[:, df.columns.str.contains("|".join(merge_keys)) | df.columns.str.contains("(min|max|median|mode|avg)", case=False)]


In [89]:
final_base_labs_without_cpr_or_pct

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_wbc,max_wbc,avg_wbc,median_wbc,min_neutrophils,max_neutrophils,avg_neutrophils,median_neutrophils,min_anc,max_anc,avg_anc,median_anc,min_lymphocytes,max_lymphocytes,avg_lymphocytes,median_lymphocytes,min_alc,max_alc,avg_alc,median_alc,min_hgb,max_hgb,avg_hgb,median_hgb,min_plt,max_plt,avg_plt,median_plt,min_glucose,max_glucose,avg_glucose,median_glucose,min_lactate,max_lactate,avg_lactate,median_lactate
0,JC1000129,131354606122,876959985,2023-04-23 22:44:00,16.9,16.9,16.90,16.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,0.8,0.8,0.8,0.14,0.14,0.14,0.14,11.8,11.8,11.80,11.8,194.0,194.0,194.0,194.0,165.0,165.0,165.0,165.0,3.56,6.44,5.05,4.42
1,JC1000129,131354606122,876959986,2023-04-23 22:44:00,16.9,16.9,16.90,16.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,0.8,0.8,0.8,0.14,0.14,0.14,0.14,11.8,11.8,11.80,11.8,194.0,194.0,194.0,194.0,165.0,165.0,165.0,165.0,3.56,6.44,5.05,4.42
2,JC1000159,131233687107,528085197,2017-06-25 16:42:00,15.4,15.4,15.40,15.4,91.5,91.5,91.5,91.5,14.10,14.10,14.10,14.10,3.1,3.1,3.1,3.1,0.48,0.48,0.48,0.48,9.2,9.2,9.20,9.2,109.0,109.0,109.0,109.0,155.0,155.0,155.0,155.0,1.50,1.70,1.60,1.50
3,JC1000159,131233687107,528085198,2017-06-25 16:42:00,15.4,15.4,15.40,15.4,91.5,91.5,91.5,91.5,14.10,14.10,14.10,14.10,3.1,3.1,3.1,3.1,0.48,0.48,0.48,0.48,9.2,9.2,9.20,9.2,109.0,109.0,109.0,109.0,155.0,155.0,155.0,155.0,1.50,1.70,1.60,1.50
4,JC1000413,131018846539,401262600,2012-04-20 05:09:00,8.9,8.9,8.90,8.9,NaN,NaN,NaN,NaN,6.61,6.61,6.61,6.61,12.8,12.8,12.8,12.8,1.14,1.14,1.14,1.14,12.0,12.0,12.00,12.0,264.0,264.0,264.0,264.0,125.0,125.0,125.0,125.0,0.70,0.70,0.70,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198010,JC999636,131263269943,592485132,2019-02-24 07:31:00,10.0,12.3,11.15,10.0,69.7,69.7,69.7,69.7,8.57,8.57,8.57,8.57,19.1,19.1,19.1,19.1,2.35,2.35,2.35,2.35,9.1,9.2,9.15,9.1,249.0,282.0,265.5,249.0,137.0,146.0,141.5,137.0,0.80,0.80,0.80,0.80
198011,JC999695,131029726519,434470924,2014-01-27 18:45:00,11.9,11.9,11.90,11.9,NaN,NaN,NaN,NaN,9.90,9.90,9.90,9.90,12.0,12.0,12.0,12.0,1.40,1.40,1.40,1.40,11.1,11.1,11.10,11.1,306.0,306.0,306.0,306.0,101.0,101.0,101.0,101.0,NaN,NaN,NaN,NaN
198012,JC999695,131029726519,434470925,2014-01-27 18:45:00,11.9,11.9,11.90,11.9,NaN,NaN,NaN,NaN,9.90,9.90,9.90,9.90,12.0,12.0,12.0,12.0,1.40,1.40,1.40,1.40,11.1,11.1,11.10,11.1,306.0,306.0,306.0,306.0,101.0,101.0,101.0,101.0,NaN,NaN,NaN,NaN
198013,JC999695,131047054269,448446821,2014-09-17 06:38:00,5.7,9.1,7.40,5.7,NaN,NaN,NaN,NaN,7.99,7.99,7.99,7.99,8.2,8.2,8.2,8.2,0.74,0.74,0.74,0.74,9.4,12.1,10.75,9.4,216.0,294.0,255.0,216.0,101.0,101.0,101.0,101.0,0.67,2.78,1.73,0.67


In [90]:

table_id_final_base_labs_without_cpr_or_pct = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_labs_without_cpr_or_pct"

# Upload the DataFrame to BigQuery
final_base_labs_without_cpr_or_pct.to_gbq(
    destination_table=table_id_final_base_labs_without_cpr_or_pct,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_labs_without_cpr_or_pct to {table_id_final_base_labs_without_cpr_or_pct}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/1458544608.py:4: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_labs_without_cpr_or_pct.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 1375.63it/s]

Uploaded final_base_labs_without_cpr_or_pct to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_labs_without_cpr_or_pct


# Urinanalysis

In [ ]:
# # old UA
# query = f"""
# WITH all_UA AS (
#     SELECT anon_id,
#          pat_enc_csn_id_coded,
#          order_time_jittered,
#          lab_name,
#          ord_value,
#          reference_unit,
#          component_id
#   FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
#   WHERE component_id IN (1230100515,1230100517,1230100518,1230100514) # check with Nick where these number from
#     AND anon_id IN (
#       SELECT DISTINCT anon_id
#       FROM {table_id_final_base_bmi_age_race_gender_adi_temp}
#     )
#     AND ord_value IS NOT NULL
# ),

# UAs_features AS (
#   SELECT
#     anon_id,
#     pat_enc_csn_id_coded,
#     order_time_jittered,

#     CASE 
#       WHEN component_id = 1230100515 AND LOWER(ord_value) IN ('negative','neg','neh') THEN 'NEGATIVE'
#       WHEN component_id = 1230100515 AND LOWER(ord_value) IN (
#         'n/a, color interference','positive','large','moderate2.0',
#         'small','trace','2+','125++','3+','4+','1+'
#       ) THEN 'POSITIVE'
#       ELSE NULL
#     END AS `Leukocyte_Esterase`,

#     CASE 
#       WHEN component_id = 1230100517 AND reference_unit LIKE '%HPF%' THEN
#         CASE 
#           WHEN SAFE_CAST(REGEXP_EXTRACT(ord_value, r'(\d+)') AS NUMERIC) >= 5 THEN 'POSITIVE'
#           WHEN SAFE_CAST(REGEXP_EXTRACT(ord_value, r'(\d+)') AS NUMERIC) < 5 THEN 'NEGATIVE'
#           ELSE NULL
#         END
#       ELSE NULL
#     END AS `WBC_urine`,

#     CASE 
#       WHEN component_id = 1230100518 AND LOWER(ord_value) IN ('none seen','no significant amount of bacteria detected.','none') THEN 'NEGATIVE'
#       WHEN component_id = 1230100518 AND LOWER(ord_value) IN ('rare','occasional','many','moderate','few','41','profuse') THEN 'POSITIVE'
#       ELSE NULL
#     END AS `Bacteria_urine`,

#     CASE 
#       WHEN component_id = 1230100514 AND LOWER(ord_value) IN ('negative','neg','neh') THEN 'NEGATIVE'
#       WHEN component_id = 1230100514 AND LOWER(ord_value) IN ('n/a, color interference','positive') THEN 'POSITIVE'
#       ELSE NULL
#     END AS `Nitrite_urine`

#   FROM all_UA
#   WHERE ord_value IS NOT NULL
# ),

# UA_cohorts AS (
#   SELECT
#     c.anon_id,
#     c.pat_enc_csn_id_coded,
#     c.order_proc_id_coded,
#     ua.Leukocyte_Esterase,
#     ua.WBC_urine,
#     ua.Bacteria_urine,
#     ua.Nitrite_urine
#   FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
#   INNER JOIN UAs_features ua
#   USING(anon_id, pat_enc_csn_id_coded)
#   WHERE TIMESTAMP_DIFF(ua.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
# )

# SELECT 
#   c.*,
#   ua.Leukocyte_Esterase,
#   ua.WBC_urine,
#   ua.Bacteria_urine,
#   ua.Nitrite_urine
# FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
# LEFT JOIN UA_cohorts ua
# USING(anon_id, pat_enc_csn_id_coded, order_proc_id_coded);
# """

# final_base_ua = client.query(query).to_dataframe()


In [101]:
query = f"""
-- UA features per blood-culture order (one row per order)
WITH base AS (
  SELECT *
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp}
),

-- limit UA table to patients in base
all_UA AS (
    SELECT anon_id,
         pat_enc_csn_id_coded,
         order_time_jittered,
         lab_name,
         ord_value,
         reference_unit,
         component_id
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE 
  component_id IN (1230100515,1230100517,1230100518,1230100514) # check with Nick where these number from
    AND 
    anon_id IN (
      SELECT DISTINCT anon_id
      FROM {table_id_final_base_bmi_age_race_gender_adi_temp}
    )
    AND ord_value IS NOT NULL
),

-- attach each UA to each blood-culture order within [-48h, 0h]
ua_window AS (
  SELECT
    b.anon_id,
    b.pat_enc_csn_id_coded,
    b.order_proc_id_coded,
    b.blood_culture_order_datetime_utc,
    u.order_time_jittered,
    u.component_id,
    u.ord_value,
    u.reference_unit
  FROM base b
  JOIN all_UA u
    USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(u.order_time_jittered,
                       b.blood_culture_order_datetime_utc,
                       HOUR) BETWEEN -48 AND 0
),

-- map raw ord_value -> POSITIVE/NEGATIVE per component
ua_labeled AS (
  SELECT
    anon_id,
    pat_enc_csn_id_coded,
    order_proc_id_coded,
    blood_culture_order_datetime_utc,
    order_time_jittered,
    component_id,

    -- Only one of these 4 will be non-NULL for a given row
    CASE 
      WHEN component_id = 1230100515 AND LOWER(ord_value) IN ('negative','neg','neh') THEN 'NEGATIVE'
      WHEN component_id = 1230100515 AND LOWER(ord_value) IN (
        'n/a, color interference','positive','large','moderate2.0',
        'small','trace','2+','125++','3+','4+','1+'
      ) THEN 'POSITIVE'
    END AS Leukocyte_Esterase,

    CASE 
      WHEN component_id = 1230100517 AND reference_unit LIKE '%HPF%' THEN
        CASE 
          WHEN SAFE_CAST(REGEXP_EXTRACT(ord_value, r'(\d+)') AS NUMERIC) >= 5 THEN 'POSITIVE'
          WHEN SAFE_CAST(REGEXP_EXTRACT(ord_value, r'(\d+)') AS NUMERIC) < 5 THEN 'NEGATIVE'
        END
    END AS WBC_urine,

    CASE 
      WHEN component_id = 1230100518 AND LOWER(ord_value) IN ('none seen','no significant amount of bacteria detected.','none') THEN 'NEGATIVE'
      WHEN component_id = 1230100518 AND LOWER(ord_value) IN ('rare','occasional','many','moderate','few','41','profuse') THEN 'POSITIVE'
    END AS Bacteria_urine,

    CASE 
      WHEN component_id = 1230100514 AND LOWER(ord_value) IN ('negative','neg','neh') THEN 'NEGATIVE'
      WHEN component_id = 1230100514 AND LOWER(ord_value) IN ('n/a, color interference','positive') THEN 'POSITIVE'
    END AS Nitrite_urine
  FROM ua_window
),

-- pick the *latest* UA per component for each order
ua_ranked AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY anon_id, pat_enc_csn_id_coded, order_proc_id_coded, component_id
      ORDER BY order_time_jittered DESC
    ) AS rn
  FROM ua_labeled
),
ua_latest AS (
  SELECT *
  FROM ua_ranked
  WHERE rn = 1
),

-- pivot/aggregate to 1 row per order
ua_pivot AS (
  SELECT
    anon_id,
    pat_enc_csn_id_coded,
    order_proc_id_coded,
    MAX(Leukocyte_Esterase) AS Leukocyte_Esterase,
    MAX(WBC_urine)         AS WBC_urine,
    MAX(Bacteria_urine)    AS Bacteria_urine,
    MAX(Nitrite_urine)     AS Nitrite_urine
  FROM ua_latest
  GROUP BY 1,2,3
)

SELECT
  b.anon_id,
  b.pat_enc_csn_id_coded,
  b.order_proc_id_coded,
  b.blood_culture_order_datetime_utc,
  u.Leukocyte_Esterase,
  u.WBC_urine,
  u.Bacteria_urine,
  u.Nitrite_urine
FROM base b
LEFT JOIN ua_pivot u
USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded);
"""
final_base_ua = client.query(query).to_dataframe()


In [102]:
final_base_ua.isnull().mean()

anon_id                             0.000000
pat_enc_csn_id_coded                0.000000
order_proc_id_coded                 0.000000
blood_culture_order_datetime_utc    0.000000
Leukocyte_Esterase                  0.409060
WBC_urine                           0.432987
Bacteria_urine                      0.409373
Nitrite_urine                       0.409070
dtype: float64

# LDA

In [95]:
query = f"""
WITH all_LDA AS (
  SELECT anon_id,
         pat_enc_csn_id_coded,
         placement_instant_jittered,
         removal_instant_jittered,
         description,
  FROM `som-nero-phi-jonc101.shc_core_2024.lda` 
  WHERE description like any ('%picc%','%ETT%','%CVC%', '%ECMO%','%EVD%','%ET%Tube%')
    or lower(description) like any ('%icu%line%','%tunnel%catheter%','%surgical%ur%catheter%','%port%','%dialysis%catheter%')                           
    AND anon_id IN (
      SELECT DISTINCT anon_id
      FROM  {table_id_final_base_bmi_age_race_gender_adi_temp}
    )
    ), 
LDA_pres as (
    select c.anon_id,
    c.pat_enc_csn_id_coded,
    c.order_proc_id_coded,
     CASE 
      WHEN description LIKE '%EVD%' THEN 'EVD'
      WHEN description LIKE '%ET%Tube%' THEN 'ET_Tube'
      WHEN LOWER(description) LIKE '%surgical%ur%catheter%' THEN 'Surgical_Urin_Catheter'
      ELSE 'otherline'
    END AS `Line_Presense`
    from all_LDA l 
    inner join  {table_id_final_base_bmi_age_race_gender_adi_temp} c using (anon_id,pat_enc_csn_id_coded)
    WHERE TIMESTAMP_DIFF(l.removal_instant_jittered, c.blood_culture_order_datetime_utc, HOUR) >0
    and TIMESTAMP_DIFF(l.placement_instant_jittered, c.blood_culture_order_datetime_utc, HOUR) < 0
)

SELECT c.anon_id,
    c.pat_enc_csn_id_coded,
    c.order_proc_id_coded,
    c.blood_culture_order_datetime_utc,
    # c.positive_blood_culture,
    IF(COUNT(lda.Line_Presense) > 0, 1, 0) as has_any_line
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c 
LEFT JOIN LDA_pres lda USING(anon_id, pat_enc_csn_id_coded, order_proc_id_coded)
GROUP BY ALL
"""

final_base_lda = client.query(query).to_dataframe()


In [97]:
final_base_lda

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,has_any_line
0,JC1483966,131019036862,401914892,2012-06-10 14:14:00,0
1,JC3701091,131393811639,1014401137,2024-12-29 03:18:00,0
2,JC3701091,131393811639,1014401136,2024-12-29 03:18:00,0
3,JC3701091,131393980589,1015147480,2025-01-03 02:08:00,0
4,JC3701091,131393980589,1015147481,2025-01-03 02:08:00,0
...,...,...,...,...,...
198010,JC6247570,131326304777,773856542,2022-01-21 18:04:00,0
198011,JC1285477,131248208163,555151400,2018-03-04 22:45:00,0
198012,JC1285477,131248208163,555151401,2018-03-04 22:45:00,0
198013,JC6525349,131361336436,901480114,2023-10-03 09:06:00,0


# merge labs, ua, and lda
# final_base_labs_without_cpr_or_pct
# final_base_ua
# final_base_lda




In [96]:
# Merge all three tables: labs, UA, and LDA
# All tables share the same merge keys
merge_keys = ["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "blood_culture_order_datetime_utc"]

# First merge labs with UA
merged_labs_ua = pd.merge(
    final_base_labs_without_cpr_or_pct, 
    final_base_ua, 
    on=merge_keys, 
    how='outer'
)

# Then merge the result with LDA
final_labs_ua_lda = pd.merge(
    merged_labs_ua, 
    final_base_lda, 
    on=merge_keys, 
    how='outer'
)

print(f"Final merged table shape: {final_labs_ua_lda.shape}")
print(f"Columns: {list(final_labs_ua_lda.columns)}")

# Display the merged table
final_labs_ua_lda


Final merged table shape: (198015, 45)
Columns: ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime_utc', 'min_wbc', 'max_wbc', 'avg_wbc', 'median_wbc', 'min_neutrophils', 'max_neutrophils', 'avg_neutrophils', 'median_neutrophils', 'min_anc', 'max_anc', 'avg_anc', 'median_anc', 'min_lymphocytes', 'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes', 'min_alc', 'max_alc', 'avg_alc', 'median_alc', 'min_hgb', 'max_hgb', 'avg_hgb', 'median_hgb', 'min_plt', 'max_plt', 'avg_plt', 'median_plt', 'min_glucose', 'max_glucose', 'avg_glucose', 'median_glucose', 'min_lactate', 'max_lactate', 'avg_lactate', 'median_lactate', 'Leukocyte_Esterase', 'WBC_urine', 'Bacteria_urine', 'Nitrite_urine', 'has_any_line']


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_wbc,max_wbc,avg_wbc,median_wbc,min_neutrophils,max_neutrophils,avg_neutrophils,median_neutrophils,min_anc,max_anc,avg_anc,median_anc,min_lymphocytes,max_lymphocytes,avg_lymphocytes,median_lymphocytes,min_alc,max_alc,avg_alc,median_alc,min_hgb,max_hgb,avg_hgb,median_hgb,min_plt,max_plt,avg_plt,median_plt,min_glucose,max_glucose,avg_glucose,median_glucose,min_lactate,max_lactate,avg_lactate,median_lactate,Leukocyte_Esterase,WBC_urine,Bacteria_urine,Nitrite_urine,has_any_line
0,JC1000129,131354606122,876959985,2023-04-23 22:44:00,16.9,16.9,16.90,16.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,0.8,0.8,0.8,0.14,0.14,0.14,0.14,11.8,11.8,11.80,11.8,194.0,194.0,194.0,194.0,165.0,165.0,165.0,165.0,3.56,6.44,5.05,4.42,POSITIVE,POSITIVE,POSITIVE,NEGATIVE,0
1,JC1000129,131354606122,876959986,2023-04-23 22:44:00,16.9,16.9,16.90,16.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,0.8,0.8,0.8,0.14,0.14,0.14,0.14,11.8,11.8,11.80,11.8,194.0,194.0,194.0,194.0,165.0,165.0,165.0,165.0,3.56,6.44,5.05,4.42,POSITIVE,POSITIVE,POSITIVE,NEGATIVE,0
2,JC1000159,131233687107,528085197,2017-06-25 16:42:00,15.4,15.4,15.40,15.4,91.5,91.5,91.5,91.5,14.10,14.10,14.10,14.10,3.1,3.1,3.1,3.1,0.48,0.48,0.48,0.48,9.2,9.2,9.20,9.2,109.0,109.0,109.0,109.0,155.0,155.0,155.0,155.0,1.50,1.70,1.60,1.50,POSITIVE,None,POSITIVE,NEGATIVE,0
3,JC1000159,131233687107,528085198,2017-06-25 16:42:00,15.4,15.4,15.40,15.4,91.5,91.5,91.5,91.5,14.10,14.10,14.10,14.10,3.1,3.1,3.1,3.1,0.48,0.48,0.48,0.48,9.2,9.2,9.20,9.2,109.0,109.0,109.0,109.0,155.0,155.0,155.0,155.0,1.50,1.70,1.60,1.50,POSITIVE,None,POSITIVE,NEGATIVE,0
4,JC1000413,131018846539,401262600,2012-04-20 05:09:00,8.9,8.9,8.90,8.9,NaN,NaN,NaN,NaN,6.61,6.61,6.61,6.61,12.8,12.8,12.8,12.8,1.14,1.14,1.14,1.14,12.0,12.0,12.00,12.0,264.0,264.0,264.0,264.0,125.0,125.0,125.0,125.0,0.70,0.70,0.70,0.70,None,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198010,JC999636,131263269943,592485132,2019-02-24 07:31:00,10.0,12.3,11.15,10.0,69.7,69.7,69.7,69.7,8.57,8.57,8.57,8.57,19.1,19.1,19.1,19.1,2.35,2.35,2.35,2.35,9.1,9.2,9.15,9.1,249.0,282.0,265.5,249.0,137.0,146.0,141.5,137.0,0.80,0.80,0.80,0.80,NEGATIVE,NEGATIVE,POSITIVE,NEGATIVE,0
198011,JC999695,131029726519,434470924,2014-01-27 18:45:00,11.9,11.9,11.90,11.9,NaN,NaN,NaN,NaN,9.90,9.90,9.90,9.90,12.0,12.0,12.0,12.0,1.40,1.40,1.40,1.40,11.1,11.1,11.10,11.1,306.0,306.0,306.0,306.0,101.0,101.0,101.0,101.0,NaN,NaN,NaN,NaN,None,None,None,None,0
198012,JC999695,131029726519,434470925,2014-01-27 18:45:00,11.9,11.9,11.90,11.9,NaN,NaN,NaN,NaN,9.90,9.90,9.90,9.90,12.0,12.0,12.0,12.0,1.40,1.40,1.40,1.40,11.1,11.1,11.10,11.1,306.0,306.0,306.0,306.0,101.0,101.0,101.0,101.0,NaN,NaN,NaN,NaN,None,None,None,None,0
198013,JC999695,131047054269,448446821,2014-09-17 06:38:00,5.7,9.1,7.40,5.7,NaN,NaN,NaN,NaN,7.99,7.99,7.99,7.99,8.2,8.2,8.2,8.2,0.74,0.74,0.74,0.74,9.4,12.1,10.75,9.4,216.0,294.0,255.0,216.0,101.0,101.0,101.0,101.0,0.67,2.78,1.73,0.67,None,None,None,None,0


In [98]:
table_id_final_labs_ua_lda = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_labs_without_cpr_or_pct_ua_lda"

# Upload the DataFrame to BigQuery
final_labs_ua_lda.to_gbq(
    destination_table=table_id_final_labs_ua_lda,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_labs_ua_lda to {table_id_final_labs_ua_lda}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/3420044538.py:4: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_labs_ua_lda.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 2737.80it/s]

Uploaded final_labs_ua_lda to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_labs_without_cpr_or_pct_ua_lda




#### the final labs_ua_lda table is stored at som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_labs_without_cpr_or_pct_ua_lda

## supplement

### creatinine 

In [99]:
table_id_final_base_bmi_age_race_gender_adi_temp = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_bmi_age_race_gender_adi_temp"
query_cr = f"""
WITH all_lab_result AS (
  SELECT anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_value, reference_unit, order_time_jittered
  FROM `som-nero-phi-jonc101.shc_core_2024.lab_result`
  WHERE anon_id IN (SELECT DISTINCT anon_id FROM {table_id_final_base_bmi_age_race_gender_adi_temp})
),
cr_raw AS (
  SELECT
    c.anon_id, c.pat_enc_csn_id_coded, c.order_proc_id_coded, c.blood_culture_order_datetime_utc,
    CASE
      WHEN LOWER(lr.base_name) = 'cr' AND LOWER(lr.reference_unit) = 'mg/dl' THEN SAFE_CAST(lr.ord_value AS FLOAT64)
    END AS cr
  FROM {table_id_final_base_bmi_age_race_gender_adi_temp} c
  LEFT JOIN all_lab_result lr USING (anon_id, pat_enc_csn_id_coded)
  WHERE TIMESTAMP_DIFF(lr.order_time_jittered, c.blood_culture_order_datetime_utc, HOUR) BETWEEN -48 AND 0
),
cr_agg AS (
  SELECT
    anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc,
    ROUND(MIN(cr),2) AS min_cr,
    ROUND(MAX(cr),2) AS max_cr,
    ROUND(AVG(cr),2) AS avg_cr,
    ROUND(APPROX_QUANTILES(cr,100)[OFFSET(50)],2) AS median_cr
  FROM cr_raw
  GROUP BY 1,2,3,4
)
SELECT b.anon_id, b.pat_enc_csn_id_coded, b.order_proc_id_coded, b.blood_culture_order_datetime_utc,
       a.min_cr, a.max_cr, a.avg_cr, a.median_cr
FROM {table_id_final_base_bmi_age_race_gender_adi_temp} b
LEFT JOIN cr_agg a USING (anon_id, pat_enc_csn_id_coded, order_proc_id_coded, blood_culture_order_datetime_utc)
ORDER BY b.anon_id, b.blood_culture_order_datetime_utc
"""
final_base_cr_only = client.query(query_cr).to_dataframe()


In [100]:
final_base_cr_only

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,min_cr,max_cr,avg_cr,median_cr
0,JC1000129,131354606122,876959985,2023-04-23 22:44:00,2.83,2.83,2.83,2.83
1,JC1000129,131354606122,876959986,2023-04-23 22:44:00,2.83,2.83,2.83,2.83
2,JC1000159,131233687107,528085198,2017-06-25 16:42:00,0.87,0.87,0.87,0.87
3,JC1000159,131233687107,528085197,2017-06-25 16:42:00,0.87,0.87,0.87,0.87
4,JC1000413,131018846539,401262600,2012-04-20 05:09:00,1.90,1.90,1.90,1.90
...,...,...,...,...,...,...,...,...
198010,JC999636,131263269943,592485132,2019-02-24 07:31:00,1.05,1.10,1.08,1.05
198011,JC999695,131029726519,434470924,2014-01-27 18:45:00,0.60,0.60,0.60,0.60
198012,JC999695,131029726519,434470925,2014-01-27 18:45:00,0.60,0.60,0.60,0.60
198013,JC999695,131047054269,448446821,2014-09-17 06:38:00,0.80,0.90,0.85,0.80


In [103]:
# Upload the DataFrame to BigQuery
table_id_final_labs_cr_only = f"{project_id}.blood_culture_stewardship_sandy_2024.final_base_labs_cr_only"
final_base_cr_only.to_gbq(
    destination_table=table_id_final_labs_cr_only,
    project_id=project_id,
    if_exists='replace'  # This will replace the table if it exists
)

print(f"Uploaded final_base_cr_only to {table_id_final_labs_cr_only}")

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_7303/2093160943.py:3: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  final_base_cr_only.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 3223.91it/s]

Uploaded final_base_cr_only to som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.final_base_labs_cr_only


In [ ]:
# Check for percentage of